In [ ]:
# from pycaret.regression import *
# from pytorch_tabnet.tab_model import TabNetRegressor
import pandas as pd
import numpy as np
import re
import ray
import shap
import torch
import optuna
import xgboost
import swifter
import itertools
from catboost import CatBoostRegressor
from sklearn import preprocessing
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
import lightgbm as lgb
from lightgbm import early_stopping,log_evaluation
from geopy.distance import great_circle, geodesic
import gc
gc.collect()

5

# 날짜분리

In [ ]:
train = pd.read_parquet("train_add_읍면동.parquet").drop(['road_in_use'], axis=1)
test = pd.read_parquet("test_add_읍면동.parquet").drop(['road_in_use'], axis=1)

In [ ]:
train.road_name.unique()

In [ ]:
test.road_name.unique()

In [ ]:
# 날짜 변수로 분리
train['base_date'] = pd.to_datetime(train['base_date'], format='%Y%m%d')
train['year'] = train.base_date.dt.year
train['month'] = train.base_date.dt.month
train['day'] = train.base_date.dt.day
train.drop(['id'], axis=1, inplace=True)

test['base_date'] = pd.to_datetime(test['base_date'], format='%Y%m%d')
test['year'] = test.base_date.dt.year
test['month'] = test.base_date.dt.month
test['day'] = test.base_date.dt.day
test.drop(['id'], axis=1, inplace=True)
test

# 기상 추가
ray tune
ptrain
ptest

In [ ]:
df1 = pd.read_csv('OBS_ASOS_DD_20221013133253.csv', encoding='euc-kr')
df2 = pd.read_csv('OBS_ASOS_DD_20221013133312.csv', encoding='euc-kr')
df1

In [ ]:
df = pd.concat([df1,df2]).fillna(0).drop(['지점','지점명'], axis=1).sort_values(by='일시').drop('기사', axis=1).reset_index(drop=True)
df['일시'] = pd.to_datetime(df['일시'])
df.to_parquet('일별제주도기상.parquet')
del df1
del df2
df

In [ ]:
df = df.set_index('일시')
df.index = pd.to_datetime(df.index)
df = df.resample(rule='10D').mean().reset_index()
df['year'] = df.일시.dt.year
df['month'] = df.일시.dt.month
df['day'] = df.일시.dt.day
df

In [ ]:
cols = df.columns[1:-3]
cols

In [ ]:
ray.init(num_cpus=16)
train[cols] = None
tmp = []
@ray.remote
def add_cols(y,m,d):
  for j in range(len(y)):
    tmp.append(df[(df.year>=y[j]-1) & (df.month>=m[j]) & (df.day > d[j]-10)].head(1)[cols].values[0])
  return tmp

train[cols] = ray.get(add_cols.remote(train.year,train.month,train.day))

In [ ]:
ray.shutdown()

In [ ]:
ray.init(num_cpus=16)
test[cols] = None
tmp = []
@ray.remote
def add_cols(y,m,d):
  for j in range(len(y)):
    tmp.append(df[(df.year>=y[j]-1) & (df.month>=m[j]) & (df.day > d[j]-10)].head(1)[cols].values[0])
  return tmp

test[cols] = ray.get(add_cols.remote(test.year,test.month,test.day))

In [ ]:
ray.shutdown()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.to_parquet('ptrain.parquet')

In [ ]:
test.to_parquet('ptest.parquet')

# 인코딩, 변수생성

In [ ]:
train = pd.read_parquet('ptrain.parquet')
test = pd.read_parquet('ptest.parquet')
train

,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,...,최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),합계 일조시간(hr),합계 일사량(MJ/m2),일 최심신적설(cm),일 최심적설(cm),평균 전운량(1/10),평균 지면온도(°C),안개 계속시간(hr)
0,2022-06-23,목,17,1,106,지방도1112호선,0,0,60.0,0.0,...,4.83,2.31,77.42,5.95,18.409,0.0,0.0,6.77,26.46,0.000
1,2022-07-28,목,21,2,103,일반국도11호선,0,0,60.0,0.0,...,5.24,2.54,77.59,5.43,17.305,0.0,0.0,7.13,30.93,0.000
2,2021-10-10,일,7,2,103,일반국도16호선,0,0,80.0,0.0,...,6.31,3.23,58.58,5.42,13.070,0.0,0.0,5.84,21.55,0.000
3,2022-03-11,금,13,2,107,태평로,0,0,50.0,0.0,...,5.66,2.70,70.99,4.93,13.217,0.0,0.0,6.25,12.58,0.000
4,2021-10-05,화,8,2,103,일반국도12호선,0,0,80.0,0.0,...,6.31,3.23,58.58,5.42,13.070,0.0,0.0,5.84,21.55,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,2021-11-04,목,16,1,107,-,0,0,50.0,0.0,...,4.78,2.35,64.00,6.66,11.723,0.0,0.0,2.84,13.66,0.000
4701213,2022-03-31,목,2,2,107,-,0,0,80.0,0.0,...,6.79,3.38,69.66,4.34,12.601,0.0,0.0,6.98,15.86,0.000
4701214,2022-06-13,월,22,2,103,일반국도12호선,0,0,60.0,0.0,...,5.14,2.29,76.94,7.72,21.644,0.0,0.0,6.23,26.84,0.933
4701215,2021-10-20,수,2,2,103,일반국도95호선,0,0,80.0,0.0,...,6.08,2.94,62.16,5.03,11.507,0.0,0.0,5.37,16.81,0.000


In [ ]:
# 이름 통합
train.loc[(train['start_node_name'] == '삼양교(서측)'), 'start_node_name'] = '삼양교'
train.loc[(train['start_node_name'] == '삼양교(동측)'), 'start_node_name'] = '삼양교'
train.loc[(train['start_node_name'] == '용흥교(동측)'), 'start_node_name'] = '용흥교'
train.loc[(train['start_node_name'] == '부록교 시종점'), 'start_node_name'] = '부록교'
train.loc[(train['start_node_name'] == '봉개교 시종점'), 'start_node_name'] = '봉개교'

train.loc[(train['end_node_name'] == '삼양교(서측)'), 'end_node_name'] = '삼양교'
train.loc[(train['end_node_name'] == '삼양교(동측)'), 'end_node_name'] = '삼양교'
train.loc[(train['end_node_name'] == '용흥교(동측)'), 'end_node_name'] = '용흥교'
train.loc[(train['end_node_name'] == '봉개교 시종점'), 'end_node_name'] = '봉개교'
train.loc[(train['end_node_name'] == '부록교 시종점'), 'end_node_name'] = '부록교'

test.loc[(test['start_node_name'] == '삼양교(서측)'), 'start_node_name'] = '삼양교'
test.loc[(test['start_node_name'] == '삼양교(동측)'), 'start_node_name'] = '삼양교'
test.loc[(test['start_node_name'] == '용흥교(동측)'), 'start_node_name'] = '용흥교'
test.loc[(test['start_node_name'] == '봉개교 시종점'), 'start_node_name'] = '봉개교'
test.loc[(test['start_node_name'] == '부록교 시종점'), 'start_node_name'] = '부록교'

test.loc[(test['end_node_name'] == '삼양교(서측)'), 'end_node_name'] = '삼양교'
test.loc[(test['end_node_name'] == '삼양교(동측)'), 'end_node_name'] = '삼양교'
test.loc[(test['end_node_name'] == '용흥교(동측)'), 'end_node_name'] = '용흥교'
test.loc[(test['end_node_name'] == '봉개교 시종점'), 'end_node_name'] = '봉개교'
test.loc[(test['end_node_name'] == '부록교 시종점'), 'end_node_name'] = '부록교'

## 라벨인코딩

In [ ]:
# 요일 라벨인코딩
dict_day = {'월' : 1, '화' : 2, '수' : 3, '목' : 4, '금' : 5, '토' : 6, '일' : 7}
func = lambda x: dict_day.get(x, x)
train['day_of_week'] = train['day_of_week'].map(func)
test['day_of_week'] = test['day_of_week'].map(func)

In [ ]:
# 시간 기준 클래스 
# 0~5
# 5~18
# 18 ~ 24

train.loc[((0 <= train['base_hour']) & (train['base_hour'] < 5)), 'hour_class'] = 1
train.loc[((5 <= train['base_hour']) & (train['base_hour'] < 18)), 'hour_class'] = 2
train.loc[((18 <= train['base_hour']) & (train['base_hour'] <= 24)), 'hour_class'] = 3

test.loc[((0 <= test['base_hour']) & (test['base_hour'] < 5)), 'hour_class'] = 1
test.loc[((5 <= test['base_hour']) & (test['base_hour'] < 18)), 'hour_class'] = 2
test.loc[((18 <= test['base_hour']) & (test['base_hour'] <= 24)), 'hour_class'] = 3

In [ ]:
# # 로드네임 라벨인코딩
le = preprocessing.LabelEncoder()
le.fit(train.road_name.unique())
train['class_road_name'] = le.transform(train['road_name'])
test['class_road_name'] = le.transform(test['road_name'])
del le

# road_name 통합
dict_node = {'일반국도12호선':1, '일반국도16호선':2, '일반국도':3, '지방도':4, '로':5, '-':6}
def convert_node(df, node_name):
    df.loc[df[node_name].str.contains('일반국도12호선', na=False), node_name] = dict_node.get('일반국도12호선')
    df.loc[df[node_name].str.contains('일반국도16호선', na=False), node_name] = dict_node.get('일반국도16호선')
    df.loc[df[node_name].str.startswith('일반국도', na=False), node_name] = dict_node.get('일반국도')
    df.loc[df[node_name].str.startswith('지방도', na=False), node_name] = dict_node.get('지방도')
    df.loc[df[node_name].str.endswith('로', na=False), node_name] = dict_node.get('로')
    df.loc[df[node_name].str.contains('-', na=False), node_name] = dict_node.get('-')
    df.loc[df[node_name].str.isalpha().notnull(), node_name] = 0

convert_node(train, 'road_name')
convert_node(test, 'road_name')

In [ ]:
# # 노드네임 concat
# train['concat_node_name'] = train['start_node_name'] + ' ' + train['end_node_name']
# test['concat_node_name'] = test['start_node_name'] + ' ' + test['end_node_name']

# le = preprocessing.LabelEncoder()
# uniq = train['concat_node_name'].unique()
# le.fit(uniq)

# train['class_concat_node_name'] = le.transform(train['concat_node_name'])
# train.drop(['concat_node_name'], axis=1, inplace=True)
# test['class_concat_node_name'] = le.transform(test['concat_node_name'])
# test.drop(['concat_node_name'], axis=1, inplace=True)
# del le

# 시작, 끝노드 라벨인코딩
le = preprocessing.LabelEncoder()
uniq = train['start_node_name'].unique().tolist() + train['end_node_name'].unique().tolist()
le.fit(uniq)

train['class_start_node_name'] = le.transform(train['start_node_name'])
train['class_end_node_name'] = le.transform(train['end_node_name'])
test['class_start_node_name'] = le.transform(test['start_node_name'])
test['class_end_node_name'] = le.transform(test['end_node_name'])

# 시작, 끝노드, 세부 분류
dict_node = {'입구':1, '교차로':2, '삼거리':3, '사거리':4, '오거리':5, '5거리':5, '교':6, '아파트':7,}
def convert_node(df, node_name):
    df.loc[df[node_name].str.contains('아파트', na=False), node_name] = dict_node.get('아파트')
    df.loc[df[node_name].str.contains('입구', na=False), node_name] = dict_node.get('입구')
    df.loc[df[node_name].str.endswith('교차로', na=False), node_name] = dict_node.get('교차로')
    df.loc[df[node_name].str.endswith('삼거리', na=False), node_name] = dict_node.get('삼거리')
    df.loc[df[node_name].str.endswith('사거리', na=False), node_name] = dict_node.get('사거리')
    df.loc[df[node_name].str.endswith('오거리', na=False), node_name] = dict_node.get('오거리')
    df.loc[df[node_name].str.endswith('5거리', na=False), node_name] = dict_node.get('5거리')
    df.loc[df[node_name].str.endswith('교', na=False), node_name] = dict_node.get('교')
    df.loc[df[node_name].isna(), node_name] = 0
    df.loc[df[node_name].str.isalpha().notnull(), node_name] = 8
    
    return 'Fin'

convert_node(train, 'start_node_name')
convert_node(train, 'end_node_name')
convert_node(test, 'start_node_name')
convert_node(test, 'end_node_name')

'Fin'

In [ ]:
# restricted 라벨인코딩
def convert_restricted(df, node_name):
    df.loc[df[node_name] == '있음', node_name] = 1
    df.loc[df[node_name] == '없음', node_name] = 0
    
    return 'Fin'

convert_restricted(train, 'start_turn_restricted')
convert_restricted(train, 'end_turn_restricted')
convert_restricted(test, 'start_turn_restricted')
convert_restricted(test, 'end_turn_restricted')

'Fin'

In [ ]:
# road_rating 라벨인코딩
le = preprocessing.LabelEncoder()
le.fit(train.road_rating.unique())
train['road_rating'] = le.transform(train['road_rating'])
test['road_rating'] = le.transform(test['road_rating'])
del le

In [ ]:
# weight_restricted 라벨인코딩
le = preprocessing.LabelEncoder()
le.fit(train.weight_restricted.unique())
train['weight_restricted'] = le.transform(train['weight_restricted'])
test['weight_restricted'] = le.transform(test['weight_restricted'])
del le

In [ ]:
train['start_node_name'] = train['start_node_name'].astype(int)
train['class_start_node_name'] = train['class_start_node_name'].astype(int)
train['start_turn_restricted'] = train['start_turn_restricted'].astype(int)
train['end_node_name'] = train['end_node_name'].astype(int)
train['class_end_node_name'] = train['class_end_node_name'].astype(int)
train['end_turn_restricted'] = train['end_turn_restricted'].astype(int)
# train['class_concat_node_name'] = train['class_concat_node_name'].astype(int)

test['start_node_name'] = test['start_node_name'].astype(int)
test['class_start_node_name'] = test['class_start_node_name'].astype(int)
test['start_turn_restricted'] = test['start_turn_restricted'].astype(int)
test['end_node_name'] = test['end_node_name'].astype(int)
test['class_end_node_name'] = test['class_end_node_name'].astype(int)
test['end_turn_restricted'] = test['end_turn_restricted'].astype(int)
# test['class_concat_node_name'] = test['class_concat_node_name'].astype(int)

## 파생변수

In [ ]:
%%time
# # 불쾌지수
# def discomfort(df):
#     df['DI'] = (df['평균기온(°C)'] + df['평균 상대습도(%)'])*0.4 + 15
#     df['DI_class'] = 0
#     df.loc[(68 < df['DI']) & (df['DI'] <= 70), 'DI_class'] = 1
#     df.loc[(70 < df['DI']) & (df['DI'] <= 75), 'DI_class'] = 2
#     df.loc[(75 < df['DI']) & (df['DI'] <= 80), 'DI_class'] = 3
#     df.loc[(83 < df['DI']) & (df['DI'] <= 86), 'DI_class'] = 4
#     df.loc[(86 < df['DI']), 'DI_class'] = 5
#     return df
# train = discomfort(train)
# test = discomfort(test)

# # 체감온도
# def bodycel(df):
#     df['bodycel'] = 13.12 + 0.6215 * df['평균기온(°C)'] - 11.37*pow(df['평균 풍속(m/s)'], 0.16) + 0.3965*pow(df['평균 풍속(m/s)'], 0.16) * df['평균기온(°C)']
#     df['bodycel_class'] = 0
#     df.loc[(-25 < df['bodycel']) & (df['bodycel'] <= -10), 'bodycel_class'] = 1
#     df.loc[(-45 < df['bodycel']) & (df['bodycel'] <= -25), 'bodycel_class'] = 2
#     df.loc[(-59 < df['bodycel']) & (df['bodycel'] <= -45), 'bodycel_class'] = 3
#     df.loc[(df['bodycel'] <= -59), 'bodycel_class'] = 4
#     return df
# train = bodycel(train)
# test = bodycel(test)

# def day_class(df):
#     df['day_class'] = None
#     df.loc[df.day <= 16, 'day_class'] = 0
#     df.loc[df.day > 16, 'day_class'] = 1
#     return df
# train = day_class(train)
# test = day_class(test)

# def is_july(df):
#     df['is_july'] = 0
#     df.loc[df.day >= 7, 'is_july'] = 1
#     return df
# train = is_july(train)
# test = is_july(test)

# Max / Lane
def derived(df):
    df['max/lane'] = round(df.maximum_speed_limit / df.lane_count, 2)
    return df
train = derived(train)
test = derived(test)

# 주말 여부
def week(df):
    df['is_week'] = 0
    df.loc[df.day_of_week.isin([6,7]), 'is_week'] = 1
    return df
train = week(train)
test = week(test)

# 출근 / 퇴근 시간 여부
def rush(df):
    df['is_rush'] = 0
    df.loc[df.base_hour.isin([7,8,9,18,19,20]), 'is_rush'] = 1
    return df
train = rush(train)
test = rush(test)

# 좌표클래스
train['tmp'] = train['start_latitude'].astype(str) + train['start_longitude'].astype(str) + train['end_latitude'].astype(str) + train['end_longitude'].astype(str)
le = preprocessing.LabelEncoder()
le.fit(train['tmp'].unique())
train['coordi_class']= le.transform(train['tmp'])
train = train.drop(['tmp'], axis=1)

test['tmp'] = test['start_latitude'].astype(str) + test['start_longitude'].astype(str) + test['end_latitude'].astype(str) + test['end_longitude'].astype(str)
test['coordi_class']= le.transform(test['tmp'])
test = test.drop(['tmp'], axis=1)
del le

# 하중 라벨인코딩
le = preprocessing.LabelEncoder()
le.fit(train['weight_restricted'].unique())
train['weight_class']= le.transform(train['weight_restricted'])
test['weight_class'] = le.transform(test['weight_restricted'])
del le

# speed limit 라벨 인코딩
le = preprocessing.LabelEncoder()
le.fit(train['maximum_speed_limit'].unique())
train['speed_class']= le.transform(train['maximum_speed_limit'])
test['speed_class'] = le.transform(test['maximum_speed_limit'])
del le

# 좌표평균
def avg_coordi(df):
    df['avg_latitude'] = (df['start_latitude'] + df['end_latitude'])/2
    df['avg_longitude'] = (df['start_longitude'] + df['end_longitude'])/2
    return df
train = avg_coordi(train)
test = avg_coordi(test)

# vincenty
train['vincenty'] = train.apply(lambda x: geodesic((x['start_latitude'], x['start_longitude']), (x['end_latitude'], x['end_longitude'])).km, axis=1)
test['vincenty'] = test.apply(lambda x: geodesic((x['start_latitude'], x['start_longitude']), (x['end_latitude'], x['end_longitude'])).km, axis=1)

# greate
train['greate'] = train.apply(lambda x: great_circle((x['start_latitude'], x['start_longitude']), (x['end_latitude'], x['end_longitude'])).km, axis=1)
test['greate'] = test.apply(lambda x: great_circle((x['start_latitude'], x['start_longitude']), (x['end_latitude'], x['end_longitude'])).km, axis=1)
gc.collect()

CPU times: user 10min 22s, sys: 6.27 s, total: 10min 28s
Wall time: 10min 28s


0

# 좌표변수 생성

In [ ]:
coo1 = pd.concat([pd.read_csv('./운행시간/train.csv')[['now_latitude', 'now_longitude']], pd.read_csv('./운행시간/train.csv')[['next_latitude', 'next_longitude']]\
                  .rename(columns = {'next_latitude':'now_latitude','next_longitude':'now_longitude'})])\
                    .drop_duplicates(keep='first').rename(columns = {'now_latitude':'latitude','now_longitude':'longitude'})
coo2 = pd.read_csv('./버스승차/train.csv')[['latitude', 'longitude']].drop_duplicates(keep='first')
coordi = pd.concat([coo1, coo2]).drop_duplicates().rename(columns = {'latitude':'위도','longitude':'경도'})
del coo1
del coo2
coordi

In [ ]:
%%time
start_train = train.apply(lambda x: (x['start_latitude'], x['start_longitude']), axis=1)
end_train = train.apply(lambda x: (x['end_latitude'], x['end_longitude']), axis=1)

start_test = test.apply(lambda x: (x['start_latitude'], x['start_longitude']), axis=1)
end_test = test.apply(lambda x: (x['end_latitude'], x['end_longitude']), axis=1)

df_coordi = coordi.apply(lambda x: (x['위도'], x['경도']), axis=1)

train['start'] = start_train
train['end'] = end_train

test['start'] = start_test
test['end'] = end_test
gc.collect()

CPU times: user 1min 14s, sys: 4.48 s, total: 1min 18s
Wall time: 1min 18s


0

In [ ]:
%%time
def cal(x):
    cnt = 0 
    for index, v in enumerate(df_coordi):
        km = great_circle(x, v).km
        if km <= 3:
            cnt += 1
    gc.collect()
    return cnt
start = list(map(lambda x: cal(x), start_train.unique()))
end = list(map(lambda x: cal(x), end_train.unique()))

CPU times: user 2min 24s, sys: 12 ms, total: 2min 24s
Wall time: 2min 24s


In [ ]:
df_match = pd.DataFrame({'start':start_train.unique(), 'match_start':start,
                        'end':end_train.unique(), 'match_end':end,})
df_match

,start,match_start,end,match_end
0,"(33.427747274683, 126.662612038652)",25,"(33.4277487730194, 126.66233499398)",25
1,"(33.5007304293026, 126.529106761554)",397,"(33.504811303868, 126.526240074258)",383
2,"(33.2791450972975, 126.368597660936)",37,"(33.280072104575, 126.362147497048)",41
3,"(33.2460808686345, 126.56720431031)",226,"(33.2455654004832, 126.56622823987)",226
4,"(33.4622143482158, 126.326551111199)",67,"(33.4626767725677, 126.330151792542)",66
...,...,...,...,...
581,"(33.4602760456265, 126.930940973848)",89,"(33.3613084756441, 126.766621876688)",28
582,"(33.4527634666688, 126.348982418565)",71,"(33.3598385219255, 126.267782917486)",61
583,"(33.2990957109828, 126.333776830927)",47,"(33.5061024917709, 126.529427823062)",375
584,"(33.2883683638233, 126.655399767109)",79,"(33.3017771487254, 126.334880977051)",41


In [ ]:
%%time
def start_func(x, col='start'):
    return df_match.loc[df_match[col] == x][f'match_{col}'].values[0]

def end_func(x, col='end'):
    return df_match.loc[df_match[col] == x][f'match_{col}'].values[0]

train['near_start_3km'] = list(map(start_func, start_train))
train['near_end_3km'] = list(map(end_func, end_train))

test['near_start_3km'] = list(map(start_func, start_test))
test['near_end_3km'] = list(map(end_func, end_test))
gc.collect()

CPU times: user 27min 25s, sys: 7.64 s, total: 27min 33s
Wall time: 27min 22s


0

# 어린이 보호 구역

In [ ]:
coordi = pd.read_csv('제주특별자치도_어린이보호구역_20220513.csv', encoding='cp949')[['위도', '경도']]
coordi.head()

,위도,경도
0,33.520859,126.585891
1,33.520859,126.585891
2,33.506991,126.537769
3,33.506991,126.537769
4,33.501856,126.465496


In [ ]:
%%time
df_coordi = coordi.apply(lambda x: (x['위도'], x['경도']), axis=1)
start = list(map(lambda x: cal(x), start_train.unique()))
end = list(map(lambda x: cal(x), end_train.unique()))

CPU times: user 2min 42s, sys: 8.9 ms, total: 2min 42s
Wall time: 2min 42s


In [ ]:
df_match = pd.DataFrame({'start':start_train.unique(), 'match_start':start,
                        'end':end_train.unique(), 'match_end':end,})
df_match

,start,match_start,end,match_end
0,"(33.427747274683, 126.662612038652)",1,"(33.4277487730194, 126.66233499398)",1
1,"(33.5007304293026, 126.529106761554)",52,"(33.504811303868, 126.526240074258)",48
2,"(33.2791450972975, 126.368597660936)",2,"(33.280072104575, 126.362147497048)",2
3,"(33.2460808686345, 126.56720431031)",21,"(33.2455654004832, 126.56622823987)",21
4,"(33.4622143482158, 126.326551111199)",6,"(33.4626767725677, 126.330151792542)",6
...,...,...,...,...
581,"(33.4602760456265, 126.930940973848)",7,"(33.3613084756441, 126.766621876688)",0
582,"(33.4527634666688, 126.348982418565)",6,"(33.3598385219255, 126.267782917486)",1
583,"(33.2990957109828, 126.333776830927)",2,"(33.5061024917709, 126.529427823062)",48
584,"(33.2883683638233, 126.655399767109)",5,"(33.3017771487254, 126.334880977051)",2


In [ ]:
%%time
def start_func(x, col='start'):
    return df_match.loc[df_match[col] == x][f'match_{col}'].values[0]

def end_func(x, col='end'):
    return df_match.loc[df_match[col] == x][f'match_{col}'].values[0]

train['child_near_start_1km'] = list(map(start_func, start_train))
train['child_near_end_1km'] = list(map(end_func, end_train))

test['child_near_start_1km'] = list(map(start_func, start_test))
test['child_near_end_1km'] = list(map(end_func, end_test))
gc.collect()

CPU times: user 27min 34s, sys: 9.32 s, total: 27min 43s
Wall time: 27min 30s


0

In [ ]:
# 특성없는데이터제거
drop_cols = []
for i in train.columns:
    nu = train[i].nunique()
    if nu <= 1:
        drop_cols.append(i)
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)
print(drop_cols)
del drop_cols
gc.collect()

['vehicle_restricted', 'height_restricted']


0

In [ ]:
train.to_parquet('coordi_train.parquet')
test.to_parquet('coordi_test.parquet')

# (안씀) 7월데이터만뽑기 

In [ ]:
train.columns

In [ ]:
seven = train[train.month == 7]
seven

In [ ]:
seven.groupby('coordi_class').target.describe().sort_values(by='count').index

In [ ]:
# 690개 안되는 클래스찾기
flaw = []
for index, values in enumerate(seven.groupby('coordi_class').target.describe().sort_values(by='count').values):
    if values[0] != 690:
        flaw.append(seven.groupby('coordi_class').target.describe().sort_values(by='count').index[index])
flaw

In [ ]:
[i for i in test.coordi_class.unique() if i not in seven.coordi_class.unique()]

In [ ]:
tmp = []
flaw2 = [] # 690개 안되는 클래스찾기
del_seven = train[train.month.isin([5,6])]
for i in tqdm(flaw):
    try:
        tmp.extend(del_seven[del_seven['coordi_class'] == i].sample(n=690-len(seven[seven['coordi_class'] == i]), replace=False, random_state=726).index.values)
    except:
        tmp.extend(del_seven[del_seven['coordi_class'] == i].sample(n=len(seven[seven['coordi_class'] == i]), replace=False, random_state=726).index.values)
    if 690-len(seven[seven['coordi_class'] == i]) != 0 or len(seven[seven['coordi_class'] == i]) != 690:
        flaw2.append(i)
df = pd.concat([seven, del_seven.loc[tmp]], axis=0).reset_index(drop=True)
gc.collect()
        
tmp2 = []
temp = train.loc[(~train.month.isin([5,6,7])), :] # 567월이 아닌 모든 데이터
for i in tqdm(flaw2):
    try:
        tmp2.extend(temp[temp['coordi_class'] == i].sample(n=690-len(df[df['coordi_class'] == i]), random_state=726, replace=False).index.values)
    except:
        tmp2.extend(temp[temp['coordi_class'] == i].sample(n=len(temp[temp['coordi_class'] == i]), random_state=726, replace=False).index.values)
df = pd.concat([df, train.loc[tmp2]], axis=0).reset_index(drop=True)
gc.collect()
        
# uniq = seven.coordi_class.unique()
# del_seven = train[train.month.isin([5,6])]
# for i in tqdm(del_seven.coordi_class.unique()):
#     if i not in uniq:
#         uniq = np.append(uniq, i)
#         try:
#             tmp.extend(del_seven[del_seven['coordi_class'] == i].sample(n=690, random_state=726, replace=False).index.values)
#         except:
#             tmp.extend(del_seven[del_seven['coordi_class'] == i].sample(n=len(del_seven[del_seven['coordi_class'] == i]), random_state=726, replace=False).index.values)
# gc.collect()

# tmp2 = []
# temp = train.loc[(~train.month.isin([5,6,7])), :]
# for i in tqdm(temp.coordi_class.unique()):
#     if i not in uniq:
#         try:
#             tmp2.extend(temp[temp['coordi_class'] == i].sample(n=690, random_state=726, replace=False).index.values)
#         except:
#             tmp2.extend(temp[temp['coordi_class'] == i].sample(n=len(temp[temp['coordi_class'] == i]), random_state=726, replace=False).index.values)  
# gc.collect()

In [ ]:
df.groupby('coordi_class').target.describe().sort_values(by='count')

In [ ]:
df.to_parquet('sampling for 7.parquet')

# (안씀) scailing 

In [ ]:
sns.lineplot(x='base_date',y='target',data=df)

In [ ]:
sns.scatterplot(x='coordi_class',y='target',data=train)

In [ ]:
sns.scatterplot(x='base_date',y='target',data=train)

In [ ]:
sns.lineplot(x='base_date',y='target',data=train[train.coordi_class==123])

In [ ]:
df.groupby('coordi_class').target.describe().reset_index()[['coordi_class', 'mean']]

In [ ]:
seven.groupby('coordi_class').target.describe().reset_index()[['coordi_class', 'mean']]

In [ ]:
uniq = df.coordi_class.unique()

In [ ]:
df.groupby('month').target.describe()

In [ ]:
df.groupby('coordi_class').target.describe()

In [ ]:
df.groupby('coordi_class').target.describe().reset_index()[['coordi_class', 'mean']]

In [ ]:
train.loc[train.month!=7, 'target'] = round(train.loc[train.month!=7, 'target']*0.7665662977297494)

In [ ]:
sns.lineplot(x='base_date',y='target',data=train)

# (안씀) 아웃라이어 제거 

In [ ]:
def outlier_iqr(data, column): 

    # lower, upper 글로벌 변수 선언하기     
    global lower, upper    
    
    # 4분위수 기준 지정하기     
    try:
        q25, q75 = np.quantile(data[column], 0.25), np.quantile(data[column], 0.75)   
    except:
        return []
    
    # IQR 계산하기     
    iqr = q75 - q25    
    
    # outlier cutoff 계산하기     
    cut_off = iqr * 1.5          
    
    # lower와 upper bound 값 구하기     
    lower, upper = q25 - cut_off, q75 + cut_off      
    
    # 1사 분위와 4사 분위에 속해있는 데이터 각각 저장하기     
    data1 = data[data[column] > upper]     
    data2 = data[data[column] < lower]
    
    return np.concatenate((data1.index.values, data2.index.values))

N = data.new_label.nunique()
for i in tqdm(range(N)):
    drop_index = outlier_iqr(data[data.new_label==i], 'target')
    if not len(drop_index):
        continue
    data = data.drop(drop_index, axis=0).reset_index(drop=True)
len(data)
del N

# (안씀) sampling 

In [ ]:
data = train[(train.year==2022) & (train.month.isin([5,6,7]))].index
sample = train[~((train.year==2022) & (train.month.isin([5,6,7])))].sample(n=len(data)//10).index
data = train.loc[list(itertools.chain(*[sample,data])), :]
data

In [ ]:
gc.collect()

In [ ]:
drop_cols = []
for i in data.columns:
    nu = data[i].nunique()
    if nu <= 1:
        drop_cols.append(i)
data.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)
print(drop_cols)
del drop_cols

In [ ]:
data.drop(['base_date'], axis=1, inplace=True)

In [ ]:
data.to_parquet('for_train.parquet', index=False)

# (안씀) 왜도 첨도 수정 

In [ ]:
# 시간 관련, 유니크 3 미만 컬럼 무시
drop_cols = ['base_date', 'base_hour','month', 'day', 'day_of_week', 'vincenty', 'greate']
for i in train.columns:
    if train[i].nunique() < 3:
        drop_cols.append(i)
drop_cols = list(set(drop_cols))
print('drop',drop_cols)
train.columns.drop(drop_cols)

In [ ]:
skew = []
kurt = []
for i in train.columns.drop(drop_cols):
    if i == 'target':
        continue
    s = train[i].skew()
    k = train[i].kurt()
    print(i)
    print(f'skew : {s}')
    print(f'kurt : {k}\n')
    if abs(s) > 2:
        skew.append(i)
    if abs(k) != 0:
        kurt.append(i)
print(f'skew : {skew}')
print(f'kurt : {kurt}')

In [ ]:
for col in skew:
    train[col] = [np.log1p(i) for i in train[col]]
    test[col] = [np.log1p(i) for i in test[col]]

if kurt:
    scaler = RobustScaler()
    scaler = scaler.fit(train[kurt])
    train[kurt] = scaler.transform(train[kurt])
    test[kurt] = scaler.transform(test[kurt])

# (안씀) 휘발유 경유 추가 

In [ ]:
gasoline = pd.read_csv('주유소_지역별_평균판매가격 휘발유.csv', encoding='cp949')
gasoline

,구분,제주
0,2022년05월01일,1967.16
1,2022년05월02일,1944.85
2,2022년05월03일,1937.59
3,2022년05월04일,1970.07
4,2022년05월05일,1975.90
...,...,...
118,2022년08월27일,1787.87
119,2022년08월28일,1787.82
120,2022년08월29일,1787.77
121,2022년08월30일,1813.60


In [ ]:
light = pd.read_csv('주유소_지역별_평균판매가격 경유.csv', encoding='cp949')
light

,구분,제주
0,2022년05월01일,2014.36
1,2022년05월02일,1999.74
2,2022년05월03일,1995.92
3,2022년05월04일,2035.42
4,2022년05월05일,2042.56
...,...,...
118,2022년08월27일,1895.67
119,2022년08월28일,1895.98
120,2022년08월29일,1895.89
121,2022년08월30일,2022.68


In [ ]:
datetime_format = "%Y년%m월%d일"
gasoline['구분'] = gasoline.apply(lambda x: datetime.strptime(x['구분'], datetime_format), axis=1)
light['구분'] = light.apply(lambda x: datetime.strptime(x['구분'], datetime_format), axis=1)

In [ ]:
gasoline.to_parquet('휘발유.parquet')
light.to_parquet('경유.parquet')

In [ ]:
%%time
def oil(x): # 휘발유
    return gasoline.loc[gasoline['구분'] == x]['제주'].values[0]

def oil2(x): # 경유
    return light.loc[light['구분'] == x]['제주'].values[0]

data['휘발유'] = list(data.base_date.apply(oil))
test['휘발유'] = list(test.base_date.apply(oil))

data['경유'] = list(data.base_date.apply(oil2))
test['경유'] = list(test.base_date.apply(oil2))
gc.collect()

CPU times: user 9min 32s, sys: 3.57 s, total: 9min 36s
Wall time: 9min 30s


0

In [ ]:
data

,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,...,near_start_3km,near_end_3km,sin_base_hour,cos_base_hour,sin_month,cos_month,sin_day_of_week,cos_day_of_week,휘발유,경유
0,2022-06-23,4,17,1,1,4,0,0,60.0,1,...,25,25,-0.997669,-0.068242,5.000000e-01,-0.866025,0.433884,-0.900969,2194.20,2220.30
1,2022-07-28,4,21,2,0,3,0,0,60.0,0,...,397,383,-0.519584,0.854419,1.224647e-16,-1.000000,0.433884,-0.900969,1908.51,2038.28
2,2022-06-23,4,11,1,0,3,0,0,60.0,0,...,8,11,0.136167,-0.990686,5.000000e-01,-0.866025,0.433884,-0.900969,2194.20,2220.30
3,2022-07-24,7,2,1,2,5,0,0,50.0,0,...,240,239,0.519584,0.854419,1.224647e-16,-1.000000,-0.781831,0.623490,1910.02,2039.23
4,2022-05-07,6,7,2,0,2,0,0,60.0,0,...,245,236,0.942261,-0.334880,8.660254e-01,-0.500000,-0.974928,-0.222521,1977.16,2043.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228536,2022-05-22,7,12,2,1,5,0,0,70.0,0,...,77,152,-0.136167,-0.990686,8.660254e-01,-0.500000,-0.781831,0.623490,2028.69,2087.08
1228537,2022-07-16,6,19,1,2,6,0,0,30.0,0,...,91,90,-0.887885,0.460065,1.224647e-16,-1.000000,-0.974928,-0.222521,2029.74,2095.57
1228538,2022-05-14,6,13,2,0,2,0,0,60.0,0,...,253,250,-0.398401,-0.917211,8.660254e-01,-0.500000,-0.974928,-0.222521,1991.61,2063.49
1228539,2022-05-31,2,2,2,2,5,0,0,50.0,0,...,232,238,0.519584,0.854419,8.660254e-01,-0.500000,0.781831,0.623490,2073.89,2046.42


In [ ]:
test

,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,...,near_start_3km,near_end_3km,sin_base_hour,cos_base_hour,sin_month,cos_month,sin_day_of_week,cos_day_of_week,휘발유,경유
0,2022-08-25,4,17,3,2,5,0,0,70.0,0,...,376,361,-9.976688e-01,-0.068242,-0.5,-0.866025,0.433884,-0.900969,1788.28,1895.99
1,2022-08-09,2,12,2,0,1,0,0,70.0,0,...,96,96,-1.361666e-01,-0.990686,-0.5,-0.866025,0.781831,0.623490,1836.42,1954.25
2,2022-08-05,5,2,1,0,2,0,0,60.0,0,...,83,77,5.195840e-01,0.854419,-0.5,-0.866025,-0.433884,-0.900969,1887.16,2012.27
3,2022-08-18,4,23,3,0,3,0,0,70.0,0,...,220,205,-2.449294e-16,1.000000,-0.5,-0.866025,0.433884,-0.900969,1790.63,1899.40
4,2022-08-10,3,17,3,1,5,0,0,70.0,0,...,236,173,-9.976688e-01,-0.068242,-0.5,-0.866025,0.974928,-0.222521,1828.36,1946.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,2022-08-27,6,5,1,0,2,0,0,80.0,0,...,180,163,9.790841e-01,0.203456,-0.5,-0.866025,-0.974928,-0.222521,1787.87,1895.67
291237,2022-08-19,5,20,2,0,3,0,0,60.0,0,...,78,87,-7.308360e-01,0.682553,-0.5,-0.866025,-0.433884,-0.900969,1788.87,1900.41
291238,2022-08-05,5,11,1,2,5,0,0,30.0,0,...,365,377,1.361666e-01,-0.990686,-0.5,-0.866025,-0.433884,-0.900969,1887.16,2012.27
291239,2022-08-12,5,7,2,2,5,0,0,60.0,0,...,159,158,9.422609e-01,-0.334880,-0.5,-0.866025,-0.433884,-0.900969,1824.36,1944.03


# (안씀) 확진자 추가 

In [ ]:
df = pd.read_csv('제주 코로나 일일 확진자.csv')
df['sum'] = df['확진자 2차'] + df['확진자 1차']
df['날짜'] = pd.to_datetime(df['날짜']) 
df['month'] = df.날짜.dt.month
df['day'] = df.날짜.dt.day
df

,날짜,확진자 2차,확진자 1차,sum,month,day
0,2022-07-31,881,1583,2464,7,31
1,2022-07-30,1223,1837,3060,7,30
2,2022-07-29,1412,1622,3034,7,29
3,2022-07-28,1196,1839,3035,7,28
4,2022-07-27,1374,2054,3428,7,27
...,...,...,...,...,...,...
117,2022-04-05,3058,2711,5769,4,5
118,2022-04-04,1783,2261,4044,4,4
119,2022-04-03,1715,3581,5296,4,3
120,2022-04-02,3119,3771,6890,4,2


In [ ]:
%%time
def covid(x): 
    return df[(df.month==x.month-1) & (df.day <= x.day)].head(1)['sum'].values[0]

data['covid'] = list(data['base_date'].apply(covid))
test['covid'] = list(test['base_date'].apply(covid))

CPU times: user 7min 9s, sys: 3.52 s, total: 7min 12s
Wall time: 7min 8s


# 시간 주기성 컬럼 생성 sin cos

In [ ]:
def add_column_cyclical_features(df, col, period, start_num):
    values = 2 * np.pi * (df[col]-start_num) / period
    kwargs = {f'sin_{col}': lambda x: np.sin(values),
              f'cos_{col}': lambda x: np.cos(values)}
    return df.assign(**kwargs)

train = add_column_cyclical_features(train,'base_hour', 23, 0)
train = add_column_cyclical_features(train,'month', 12, 1)
train = add_column_cyclical_features(train,'day_of_week', 7, 1)
train.loc[train.month.isin([1,3,5,7,8,10,12])] = add_column_cyclical_features(train.loc[train.month.isin([1,3,5,7,8,10,12])],'day', 31, 1)
train.loc[train.month.isin([4,6,9,11])] = add_column_cyclical_features(train.loc[train.month.isin([4,6,9,11])],'day', 30, 1)
train.loc[train.month==2] = add_column_cyclical_features(train.loc[train.month==2],'day', 28, 1)

test = add_column_cyclical_features(test,'base_hour', 23, 0)
test = add_column_cyclical_features(test,'month', 12, 1)
test = add_column_cyclical_features(test,'day_of_week', 7, 1)
test.loc[test.month.isin([1,3,5,7,8,10,12])] = add_column_cyclical_features(test.loc[test.month.isin([1,3,5,7,8,10,12])],'day', 31, 1)
test.loc[test.month.isin([4,6,9,11])] = add_column_cyclical_features(test.loc[test.month.isin([4,6,9,11])],'day', 30, 1)
test.loc[test.month==2] = add_column_cyclical_features(test.loc[test.month==2],'day', 28, 1)
gc.collect()

0

In [ ]:
train.to_parquet('for_train.parquet')
test.to_parquet('for_test.parquet')

# 567월 sampling
trian -> data

In [ ]:
# 전처리 완료된거
train = pd.read_parquet('for_train.parquet')
test = pd.read_parquet('for_test.parquet')

In [ ]:
# data = train[(train['year']==2022) & (train['month'] >= 7)].reset_index(drop=True)
data = train[(train['month'].isin([5,6,7]))].reset_index(drop=True)
data

# target 관련 변수 생성

### coordi_cls

In [ ]:
df = data.groupby(['coordi_class', 'base_hour']).target.describe().reset_index()
df

In [ ]:
%%time
def df_avg(x,y):
    return df.loc[(df['coordi_class'] == x) & (df['base_hour'] == y)]['mean'].values[0]

def df_std(x,y):
    return df.loc[(df['coordi_class'] == x) & (df['base_hour'] == y)]['std'].values[0]

data['coordi_hour_target_avg'] = list(data.apply(lambda x: df_avg(x.coordi_class, x.base_hour), axis=1))
data['coordi_hour_target_std'] = list(data.apply(lambda x: df_std(x.coordi_class, x.base_hour), axis=1))

test['coordi_hour_target_avg'] = list(test.apply(lambda x: df_avg(x.coordi_class, x.base_hour), axis=1))
test['coordi_hour_target_std'] = list(test.apply(lambda x: df_std(x.coordi_class, x.base_hour), axis=1))
gc.collect()

CPU times: user 12min 34s, sys: 1.98 s, total: 12min 36s
Wall time: 12min 36s


0

In [ ]:
data = data.fillna(1e-8)
test = test.fillna(1e-8)

In [ ]:
data['derived_0'] = data['near_end_3km'] * data['maximum_speed_limit']
test['derived_0'] = test['near_end_3km'] * test['maximum_speed_limit']

data['derived_target_0'] = data['coordi_hour_target_avg'] / data['lane_count']
test['derived_target_0'] = test['coordi_hour_target_avg'] / test['lane_count']

data['derived_target_1'] = data['max/lane'] * data['coordi_hour_target_std']
test['derived_target_1'] = test['max/lane'] * test['coordi_hour_target_std']

data['derived_target_2'] = data['near_end_3km'] * (data['coordi_hour_target_std'] + 1e-8)
test['derived_target_2'] = test['near_end_3km'] * (test['coordi_hour_target_std'] + 1e-8)

data['derived_target_3'] = data['coordi_hour_target_avg'] * data['lane_count']
test['derived_target_3'] = test['coordi_hour_target_avg'] * test['lane_count']

data['derived_target_4'] = data['max/lane'] / (data['coordi_hour_target_std'] + 1e-8)
test['derived_target_4'] = test['max/lane'] / (test['coordi_hour_target_std'] + 1e-8)

data['derived_target_5'] = data['coordi_hour_target_std'] / data['lane_count']
test['derived_target_5'] = test['coordi_hour_target_std'] / test['lane_count']

data['derived_target_6'] = data['maximum_speed_limit'] / (data['coordi_hour_target_std'] + 1e-8)
test['derived_target_6'] = test['maximum_speed_limit'] / (test['coordi_hour_target_std'] + 1e-8)

data['derived_target_7'] = data['maximum_speed_limit'] * (data['coordi_hour_target_std'] + 1e-8)
test['derived_target_7'] = test['maximum_speed_limit'] * (test['coordi_hour_target_std'] + 1e-8)

### cls_road_name

In [ ]:
df = data.groupby(['class_road_name', 'base_hour']).target.describe().reset_index()
df

In [ ]:
%%time
def df_avg(x,y):
    return df.loc[(df['class_road_name'] == x) & (df['base_hour'] == y)]['mean'].values[0]

def df_std(x,y):
    return df.loc[(df['class_road_name'] == x) & (df['base_hour'] == y)]['std'].values[0]

data['road_hour_target_avg'] = list(data.apply(lambda x: df_avg(x.class_road_name, x.base_hour), axis=1))
data['road_hour_target_std'] = list(data.apply(lambda x: df_std(x.class_road_name, x.base_hour), axis=1))

test['road_hour_target_avg'] = list(test.apply(lambda x: df_avg(x.class_road_name, x.base_hour), axis=1))
test['road_hour_target_std'] = list(test.apply(lambda x: df_std(x.class_road_name, x.base_hour), axis=1))
gc.collect()

CPU times: user 11min 49s, sys: 2.78 s, total: 11min 52s
Wall time: 11min 52s


0

In [ ]:
data = data.fillna(1e-8)
test = test.fillna(1e-8)

In [ ]:
data['derived_target_8'] = data['road_hour_target_avg'] / data['lane_count']
test['derived_target_8'] = test['road_hour_target_avg'] / test['lane_count']

data['derived_target_9'] = data['max/lane'] * (data['road_hour_target_std'] + 1e-8)
test['derived_target_9'] = test['max/lane'] * (test['road_hour_target_std'] + 1e-8)

data['derived_target_10'] = data['near_end_3km'] * (data['road_hour_target_std'] + 1e-8)
test['derived_target_10'] = test['near_end_3km'] * (test['road_hour_target_std'] + 1e-8)

data['derived_target_11'] = data['road_hour_target_avg'] * data['lane_count']
test['derived_target_11'] = test['road_hour_target_avg'] * test['lane_count']

data['derived_target_12'] = data['max/lane'] / (data['road_hour_target_std'] + 1e-8)
test['derived_target_12'] = test['max/lane'] / (test['road_hour_target_std'] + 1e-8)

data['derived_target_13'] = data['road_hour_target_std'] / data['lane_count']
test['derived_target_13'] = test['road_hour_target_std'] / test['lane_count']

data['derived_target_14'] = data['maximum_speed_limit'] / (data['road_hour_target_std'] + 1e-8)
test['derived_target_14'] = test['maximum_speed_limit'] / (test['road_hour_target_std'] + 1e-8)

data['derived_target_15'] = data['maximum_speed_limit'] * (data['road_hour_target_std'] + 1e-8)
test['derived_target_15'] = test['maximum_speed_limit'] * (test['road_hour_target_std'] + 1e-8)

In [ ]:
data.to_parquet('train 567.parquet')
test.to_parquet('test 567.parquet')

# Baseline XGB

In [ ]:
data = pd.read_parquet('train 567.parquet')
test = pd.read_parquet('test 567.parquet')

In [ ]:
train_cols = data.columns.drop(['target', 'base_date','multi_linked', 'connect_code','speed_class', 'year', 'is_week',
       'weight_class', '일 최심신적설(cm)', '안개 계속시간(hr)', 'cos_month', 'sin_month',
       '합계 일사량(MJ/m2)', '일 최심적설(cm)', 'coordi_hour_target_avg', 'derived_target_0', 'derived_target_3', 'start', 'end',
       'child_near_start_1km', 'child_near_end_1km'])
len(train_cols), train_cols

(62,
 Index(['day_of_week', 'base_hour', 'lane_count', 'road_rating', 'road_name',
        'maximum_speed_limit', 'weight_restricted', 'road_type',
        'start_node_name', 'start_latitude', 'start_longitude',
        'start_turn_restricted', 'end_node_name', 'end_latitude',
        'end_longitude', 'end_turn_restricted', 'city_id', 'month', 'day',
        '평균기온(°C)', '일강수량(mm)', '최대 풍속(m/s)', '평균 풍속(m/s)', '평균 상대습도(%)',
        '합계 일조시간(hr)', '평균 전운량(1/10)', '평균 지면온도(°C)', 'hour_class',
        'class_road_name', 'class_start_node_name', 'class_end_node_name',
        'max/lane', 'is_rush', 'coordi_class', 'avg_latitude', 'avg_longitude',
        'vincenty', 'greate', 'near_start_3km', 'near_end_3km', 'sin_base_hour',
        'cos_base_hour', 'sin_day_of_week', 'cos_day_of_week',
        'coordi_hour_target_std', 'derived_0', 'derived_target_1',
        'derived_target_2', 'derived_target_4', 'derived_target_5',
        'derived_target_6', 'derived_target_7', 'road_hour_target_avg',

In [ ]:
# 기존 best
train_cols = ['end_longitude', 'maximum_speed_limit',
       'near_end_3km', 'base_hour', 'start_latitude',
       'end_latitude', 'road_rating', 'start_longitude',
       'road_name', 'end_node_name', 'class_end_node_name', 
       'class_start_node_name', 'max/lane', 'coordi_class', 'road_type',
       'cos_base_hour', 'weight_restricted', 'sin_base_hour', 'month',
       'lane_count', 'start_node_name', 'greate', 'day_of_week',]
# [99]	validation_0-mae:3.43180

In [ ]:
# 최근 best
train_cols = ['end_longitude', 'maximum_speed_limit',
       'near_end_3km', 'base_hour', 'start_latitude',
       'end_latitude', 'road_rating', 'start_longitude',
       'road_name', 'end_node_name', 'class_end_node_name', 
       'class_start_node_name', 'max/lane', 'coordi_class', 'road_type',
       'cos_base_hour', 'weight_restricted', 'sin_base_hour', 'month',
       'lane_count', 'start_node_name', 'greate', 'day_of_week','coordi_hour_target_std',
       'derived_target_1','derived_target_2', 'derived_0']

In [ ]:
# 구더기
train_cols = ['end_longitude', 'maximum_speed_limit',
       'near_end_3km', 'base_hour', 'start_latitude',
       'end_latitude', 'road_rating', 'start_longitude',
       'road_name', 'end_node_name', 'class_end_node_name', 
       'class_start_node_name', 'max/lane', 'coordi_class', 'road_type',
       'cos_base_hour', 'weight_restricted', 'sin_base_hour', 'month',
       'lane_count', 'start_node_name', 'greate', 'day_of_week',
       'derived_target_11', 'derived_target_8', 'derived_target_10', 'derived_target_12', 'derived_target_2']

In [ ]:
p = {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 726, 'early_stopping_rounds': 10, 'max_depth': 16, 'min_child_weight': 55, 'learning_rate': 0.08075497353245195, 'colsample_bytree': 0.9, 'lambda': 0.0024542265981280726, 'alpha': 0.020952494463469185, 'gamma': 0.0029765534687278272, 'subsample': 0.9950145824699698}
X_train, X_val, y_train, y_val = train_test_split(data[train_cols], data.target, test_size=0.2, random_state=726)
xgb = xgboost.XGBRegressor(**p)
xgb.fit(X_train,y_train, eval_set=[(X_val, y_val)], eval_metric = 'mae')
xgb.predict(test[train_cols])

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.


[0]	validation_0-mae:37.39680
[1]	validation_0-mae:34.37804
[2]	validation_0-mae:31.60460
[3]	validation_0-mae:29.05649
[4]	validation_0-mae:26.71527
[5]	validation_0-mae:24.56541
[6]	validation_0-mae:22.59152
[7]	validation_0-mae:20.77937
[8]	validation_0-mae:19.11696
[9]	validation_0-mae:17.59414
[10]	validation_0-mae:16.19870
[11]	validation_0-mae:14.92128
[12]	validation_0-mae:13.75252
[13]	validation_0-mae:12.68454
[14]	validation_0-mae:11.71038
[15]	validation_0-mae:10.82263
[16]	validation_0-mae:10.01421
[17]	validation_0-mae:9.28068
[18]	validation_0-mae:8.61533
[19]	validation_0-mae:8.01132
[20]	validation_0-mae:7.46564
[21]	validation_0-mae:6.97290
[22]	validation_0-mae:6.52825
[23]	validation_0-mae:6.12831
[24]	validation_0-mae:5.76981
[25]	validation_0-mae:5.44803
[26]	validation_0-mae:5.16058
[27]	validation_0-mae:4.90323
[28]	validation_0-mae:4.67474
[29]	validation_0-mae:4.47115
[30]	validation_0-mae:4.29015
[31]	validation_0-mae:4.13001
[32]	validation_0-mae:3.98850
[33

array([25.781765, 42.06639 , 66.43037 , ..., 22.240036, 21.851263,
       48.331455], dtype=float32)

# 모델링

## Permutaion Importance

In [ ]:
pd.set_option('display.max_columns',10)
pd.set_option('display.max_row',10)

In [ ]:
%%time
# fix_model : 훈련된 모델
# X_train : 훈련데이터 Feature
# y_train : 훈련데이터 Target
# scoring : Feature 를 Shuffler 한 뒤, 예측값과 실제값을 어떤 Metric 을 사용해 비교할지
# n_repeats : 특정 Feature 를 몇번 Shuffle 할 것인지
# random_state : 난수 고정
result = permutation_importance(xgb, 
                                data[train_cols], 
                                data.target.values, 
                                scoring = make_scorer(mean_absolute_error, greater_is_better=False),
                                n_repeats=10,
                                random_state=726)

# Feature label
Feature = data[train_cols]

# Feature 중요도를 오름차순으로 정렬한 뒤, 해당 Feature 의 index 를 저장
sorted_result = result.importances_mean.argsort()

# 결과를 DataFrame 화
importances = pd.DataFrame(result.importances_mean[sorted_result], index=Feature.columns[sorted_result]).sort_values(0, ascending=False)   

CPU times: user 20min 24s, sys: 24 s, total: 20min 48s
Wall time: 20min 48s


In [ ]:
pd.set_option('display.max_row',None)
importances

,0
road_hour_target_avg,5.503581
near_start_3km,2.374845
vincenty,1.061783
end_longitude,1.033342
near_end_3km,1.000320
derived_target_15,0.924376
start_latitude,0.822747
derived_target_10,0.678639
start_longitude,0.610356
end_node_name,0.525035


In [ ]:
importances.index

Index(['road_hour_target_avg', 'near_start_3km', 'vincenty', 'end_longitude',
       'near_end_3km', 'derived_target_15', 'start_latitude',
       'derived_target_10', 'start_longitude', 'end_node_name', 'end_latitude',
       'class_end_node_name', 'derived_target_11', 'derived_0',
       'class_start_node_name', 'avg_longitude', 'maximum_speed_limit',
       'derived_target_2', 'avg_latitude', 'derived_target_12',
       'weight_restricted', 'base_hour', 'start_node_name', 'city_id',
       'class_road_name', 'derived_target_14', 'derived_target_8',
       'derived_target_1', 'coordi_class', 'derived_target_4',
       'derived_target_9', 'road_name', 'derived_target_5',
       'coordi_hour_target_std', 'road_rating', 'cos_base_hour',
       'sin_base_hour', 'month', 'max/lane', 'road_hour_target_std',
       'derived_target_7', 'day_of_week', 'derived_target_6',
       'derived_target_13', 'start_turn_restricted', 'lane_count', 'greate',
       'road_type', 'end_turn_restricted', 'da

## Tune XGB

In [ ]:
train_cols

['end_longitude',
 'maximum_speed_limit',
 'near_end_3km',
 'base_hour',
 'start_latitude',
 'end_latitude',
 'road_rating',
 'start_longitude',
 'road_name',
 'end_node_name',
 'class_end_node_name',
 'class_start_node_name',
 'max/lane',
 'coordi_class',
 'road_type',
 'cos_base_hour',
 'weight_restricted',
 'sin_base_hour',
 'month',
 'lane_count',
 'start_node_name',
 'greate',
 'day_of_week',
 'coordi_hour_target_std',
 'derived_target_1',
 'derived_target_2',
 'derived_0']

In [ ]:
def objectiveXGB(trial,X,y,seed):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 3000, 3000),
        'tree_method': trial.suggest_categorical("tree_method", ["gpu_hist"]),
        'gpu_id': trial.suggest_int('gpu_id', 0, 0),
        'predictor': trial.suggest_categorical("predictor", ["gpu_predictor"]),
        'random_state': trial.suggest_int('random_state', seed, seed),
        'early_stopping_rounds' : trial.suggest_int('early_stopping_rounds', 10, 10),
        'max_depth':trial.suggest_int('max_depth', 15, 25),
        'min_child_weight':trial.suggest_int('min_child_weight', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate',1e-5, 1e-1),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1, 1, 0.1),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'subsample': trial.suggest_loguniform('subsample', 1e-1, 1),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = xgboost.XGBRegressor(**param)
    xgb = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)],)
    score = mean_absolute_error(xgb.predict(X_test), y_test)
    gc.collect()
    return score

studyXGB=optuna.create_study(direction='minimize')
studyXGB.optimize(lambda trial: objectiveXGB(trial, data[train_cols], data.target, seed=726), n_trials=100)

[I 2022-11-09 17:25:22,874] A new study created in memory with name: no-name-c36905bd-b5b2-41e8-ae26-998d9fd2ac20
[I 2022-11-09 17:26:32,126] Trial 0 finished with value: 3.007445504894964 and parameters: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 726, 'early_stopping_rounds': 10, 'max_depth': 22, 'min_child_weight': 81, 'learning_rate': 0.05650680411086892, 'colsample_bytree': 0.4, 'lambda': 0.0429631648440273, 'alpha': 0.20970941150349823, 'gamma': 0.0014014184090552614, 'subsample': 0.12020546123494093}. Best is trial 0 with value: 3.007445504894964.
[I 2022-11-09 17:27:55,826] Trial 1 finished with value: 2.9679025164332034 and parameters: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 726, 'early_stopping_rounds': 10, 'max_depth': 23, 'min_child_weight': 269, 'learning_rate': 0.05669387208787173, 'colsample_bytree': 0.7000000000000001, 'lambda': 0.0985182

In [ ]:
print('study.best_params:', studyXGB.best_trial.value)
print('Number of finished trials:', len(studyXGB.trials))
print('Best trial:', studyXGB.best_trial.params)
print('study.best_params:', studyXGB.best_params)

study.best_params: 2.918427850001272
Number of finished trials: 100
Best trial: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 726, 'early_stopping_rounds': 10, 'max_depth': 16, 'min_child_weight': 55, 'learning_rate': 0.08075497353245195, 'colsample_bytree': 0.9, 'lambda': 0.0024542265981280726, 'alpha': 0.020952494463469185, 'gamma': 0.0029765534687278272, 'subsample': 0.9950145824699698}
study.best_params: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 726, 'early_stopping_rounds': 10, 'max_depth': 16, 'min_child_weight': 55, 'learning_rate': 0.08075497353245195, 'colsample_bytree': 0.9, 'lambda': 0.0024542265981280726, 'alpha': 0.020952494463469185, 'gamma': 0.0029765534687278272, 'subsample': 0.9950145824699698}


In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=726)
skf = StratifiedKFold(n_splits=5, random_state=726)
mae = []
models = []
p = {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 726, 'early_stopping_rounds': 10, 'max_depth': 16, 'min_child_weight': 55, 'learning_rate': 0.08075497353245195, 'colsample_bytree': 0.9, 'lambda': 0.0024542265981280726, 'alpha': 0.020952494463469185, 'gamma': 0.0029765534687278272, 'subsample': 0.9950145824699698}
# p = studyXGB.best_params
le = preprocessing.LabelEncoder()
y = le.fit_transform(data.road_name)
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
# for step, (train_idx, val_idx) in enumerate(skf.split(data[train_cols], y)):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    models.append(xgboost.XGBRegressor(**p,eval_metric='mae'))
    models[-1] = models[-1].fit(X_train, y_train, eval_set=[(X_val, y_val)])
    pred = models[-1].predict(X_val)
    mae.append(mean_absolute_error(pred, y_val))
    gc.collect()
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

0
[0]	validation_0-mae:37.39779
[1]	validation_0-mae:34.37923
[2]	validation_0-mae:31.60493
[3]	validation_0-mae:29.05593
[4]	validation_0-mae:26.71466
[5]	validation_0-mae:24.56417
[6]	validation_0-mae:22.58974
[7]	validation_0-mae:20.77807
[8]	validation_0-mae:19.11614
[9]	validation_0-mae:17.59345
[10]	validation_0-mae:16.19845
[11]	validation_0-mae:14.92160
[12]	validation_0-mae:13.75418
[13]	validation_0-mae:12.68708
[14]	validation_0-mae:11.71318
[15]	validation_0-mae:10.82561
[16]	validation_0-mae:10.01841
[17]	validation_0-mae:9.28469
[18]	validation_0-mae:8.61943
[19]	validation_0-mae:8.01649
[20]	validation_0-mae:7.47229
[21]	validation_0-mae:6.97957
[22]	validation_0-mae:6.53647
[23]	validation_0-mae:6.13714
[24]	validation_0-mae:5.77808
[25]	validation_0-mae:5.45714
[26]	validation_0-mae:5.16851
[27]	validation_0-mae:4.91062
[28]	validation_0-mae:4.68195
[29]	validation_0-mae:4.47769
[30]	validation_0-mae:4.29559
[31]	validation_0-mae:4.13563
[32]	validation_0-mae:3.99357
[

In [ ]:
def objectiveXGB(trial,X,y,seed):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 3000, 3000),
        'tree_method': trial.suggest_categorical("tree_method", ["gpu_hist"]),
        'gpu_id': trial.suggest_int('gpu_id', 0, 0),
        'predictor': trial.suggest_categorical("predictor", ["gpu_predictor"]),
        'random_state': trial.suggest_int('random_state', seed, seed),
        'early_stopping_rounds' : trial.suggest_int('early_stopping_rounds', 10, 10),
        'max_depth':trial.suggest_int('max_depth', 15, 25),
        'min_child_weight':trial.suggest_int('min_child_weight', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate',1e-5, 1e-1),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1, 1, 0.1),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'subsample': trial.suggest_loguniform('subsample', 1e-1, 1),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = xgboost.XGBRegressor(**param)
    xgb = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)],)
    score = mean_absolute_error(xgb.predict(X_test), y_test)
    gc.collect()
    return score

studyXGB=optuna.create_study(direction='minimize')
studyXGB.optimize(lambda trial: objectiveXGB(trial, data[train_cols], data.target, seed=1004), n_trials=100)

[I 2022-11-09 19:32:11,509] A new study created in memory with name: no-name-efb0cb51-8c78-4486-85e2-159610108235
[I 2022-11-09 19:38:54,680] Trial 0 finished with value: 2.9710961279644295 and parameters: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 1004, 'early_stopping_rounds': 10, 'max_depth': 24, 'min_child_weight': 128, 'learning_rate': 0.013875321196309465, 'colsample_bytree': 0.7000000000000001, 'lambda': 0.08069888878038874, 'alpha': 0.08167872669014886, 'gamma': 0.0038714470244355613, 'subsample': 0.21742744977884954}. Best is trial 0 with value: 2.9710961279644295.
[I 2022-11-09 19:39:53,954] Trial 1 finished with value: 2.935732618548898 and parameters: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 1004, 'early_stopping_rounds': 10, 'max_depth': 16, 'min_child_weight': 32, 'learning_rate': 0.06244460150367794, 'colsample_bytree': 0.5, 'lambda': 0.00

In [ ]:
print('study.best_params:', studyXGB.best_trial.value)
print('Number of finished trials:', len(studyXGB.trials))
print('Best trial:', studyXGB.best_trial.params)
print('study.best_params:', studyXGB.best_params)

study.best_params: 2.9095170566855573
Number of finished trials: 100
Best trial: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 1004, 'early_stopping_rounds': 10, 'max_depth': 18, 'min_child_weight': 81, 'learning_rate': 0.015689823014064068, 'colsample_bytree': 1.0, 'lambda': 2.1099124749293514, 'alpha': 0.04075021450770544, 'gamma': 0.01142980650497577, 'subsample': 0.97250947484207}
study.best_params: {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 1004, 'early_stopping_rounds': 10, 'max_depth': 18, 'min_child_weight': 81, 'learning_rate': 0.015689823014064068, 'colsample_bytree': 1.0, 'lambda': 2.1099124749293514, 'alpha': 0.04075021450770544, 'gamma': 0.01142980650497577, 'subsample': 0.97250947484207}


In [ ]:
p = {'n_estimators': 3000, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor', 'random_state': 1004, 'early_stopping_rounds': 10, 'max_depth': 18, 'min_child_weight': 81, 'learning_rate': 0.015689823014064068, 'colsample_bytree': 1.0, 'lambda': 2.1099124749293514, 'alpha': 0.04075021450770544, 'gamma': 0.01142980650497577, 'subsample': 0.97250947484207}
# p = studyXGB.best_params
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    models.append(xgboost.XGBRegressor(**p,eval_metric='mae'))
    models[-1] = models[-1].fit(X_train, y_train, eval_set=[(X_val, y_val)])
    pred = models[-1].predict(X_val)
    mae.append(mean_absolute_error(pred, y_val))
    gc.collect()
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

0
[0]	validation_0-mae:40.04482
[1]	validation_0-mae:39.41820
[2]	validation_0-mae:38.80139
[3]	validation_0-mae:38.19429
[4]	validation_0-mae:37.59667
[5]	validation_0-mae:37.00842
[6]	validation_0-mae:36.42942
[7]	validation_0-mae:35.85958
[8]	validation_0-mae:35.29872
[9]	validation_0-mae:34.74663
[10]	validation_0-mae:34.20325
[11]	validation_0-mae:33.66840
[12]	validation_0-mae:33.14197
[13]	validation_0-mae:32.62382
[14]	validation_0-mae:32.11378
[15]	validation_0-mae:31.61177
[16]	validation_0-mae:31.11765
[17]	validation_0-mae:30.63134
[18]	validation_0-mae:30.15271
[19]	validation_0-mae:29.68164
[20]	validation_0-mae:29.21803
[21]	validation_0-mae:28.76173
[22]	validation_0-mae:28.31268
[23]	validation_0-mae:27.87068
[24]	validation_0-mae:27.43571
[25]	validation_0-mae:27.00766
[26]	validation_0-mae:26.58641
[27]	validation_0-mae:26.17186
[28]	validation_0-mae:25.76381
[29]	validation_0-mae:25.36224
[30]	validation_0-mae:24.96705
[31]	validation_0-mae:24.57814
[32]	validation_

## Tune Cat

In [ ]:
def objectiveCAT(trial,X,y,seed):
    param = {
        'silent':True,
        'task_type':trial.suggest_categorical("task_type", ["GPU"]),
        "iterations": trial.suggest_int('iterations', 4000, 4000),
        'random_state': trial.suggest_int('random_state', seed, seed),
        'od_wait': trial.suggest_int('od_wait', 10, 10),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-4, 1),
        'depth': trial.suggest_int('depth', 5, 16),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,300),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1, 15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = CatBoostRegressor(**param)
    cat = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)], early_stopping_rounds=10)
    score = mean_absolute_error(cat.predict(X_test), y_test)
    gc.collect()
    return score

studyCAT=optuna.create_study(direction='minimize')
studyCAT.optimize(lambda trial: objectiveCAT(trial, data[train_cols], data.target, seed=726), n_trials=100)

[I 2022-11-10 02:44:49,583] A new study created in memory with name: no-name-4e9528d6-3031-473c-be22-5a9eb44d89b4
[I 2022-11-10 02:45:29,814] Trial 0 finished with value: 2.963109735026338 and parameters: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 726, 'od_wait': 10, 'learning_rate': 0.34333804241834237, 'depth': 16, 'reg_lambda': 46.39432381808148, 'random_strength': 35.35370760839483, 'min_data_in_leaf': 195, 'leaf_estimation_iterations': 1, 'bagging_temperature': 0.09550974790349201}. Best is trial 0 with value: 2.963109735026338.
[I 2022-11-10 02:46:01,183] Trial 1 finished with value: 3.9515495129233846 and parameters: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 726, 'od_wait': 10, 'learning_rate': 0.01079340815339038, 'depth': 7, 'reg_lambda': 18.761339602668542, 'random_strength': 16.412852797981486, 'min_data_in_leaf': 110, 'leaf_estimation_iterations': 5, 'bagging_temperature': 4.648120253597642}. Best is trial 0 with value: 2.963109735026338.
[I 202

In [ ]:
print('study.best_params:', studyCAT.best_trial.value)
print('Number of finished trials:', len(studyCAT.trials))
print('Best trial:', studyCAT.best_trial.params)
print('study.best_params:', studyCAT.best_params)

study.best_params: 2.9386772093130045
Number of finished trials: 100
Best trial: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 726, 'od_wait': 10, 'learning_rate': 0.0446744999184302, 'depth': 13, 'reg_lambda': 71.0418363483577, 'random_strength': 40.72423744381926, 'min_data_in_leaf': 264, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.024717029972342392}
study.best_params: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 726, 'od_wait': 10, 'learning_rate': 0.0446744999184302, 'depth': 13, 'reg_lambda': 71.0418363483577, 'random_strength': 40.72423744381926, 'min_data_in_leaf': 264, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.024717029972342392}


In [ ]:
mae = []
models2 = []
p = {'task_type': 'GPU', 'iterations': 4000, 'random_state': 726, 'od_wait': 10, 'learning_rate': 0.0446744999184302, 'depth': 13, 'reg_lambda': 71.0418363483577, 'random_strength': 40.72423744381926, 'min_data_in_leaf': 264, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.024717029972342392}
# p = studyCAT.best_params
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    models2.append(CatBoostRegressor(**p, silent=True, eval_metric='MAE'))
    models2[-1] = models2[-1].fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=5)
    pred = models2[-1].predict(X_val)
    mae.append(mean_absolute_error(pred, y_val))
    gc.collect()
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

0


Default metric period is 5 because MAE is/are not implemented for GPU


1


Default metric period is 5 because MAE is/are not implemented for GPU


2


...

3


Default metric period is 5 because MAE is/are not implemented for GPU


4


Default metric period is 5 because MAE is/are not implemented for GPU


분할의 MAE : [2.9347249698132294, 2.9155463018165593, 2.921472532716972, 2.9239239296433044, 2.9314708042244644]
평균 MAE : 2.925427707642906


In [ ]:
def objectiveCAT(trial,X,y,seed):
    param = {
        'silent':True,
        'task_type':trial.suggest_categorical("task_type", ["GPU"]),
        "iterations": trial.suggest_int('iterations', 4000, 4000),
        'random_state': trial.suggest_int('random_state', seed, seed),
        'od_wait': trial.suggest_int('od_wait', 10, 10),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-4, 1),
        'depth': trial.suggest_int('depth', 5, 16),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,300),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1, 15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = CatBoostRegressor(**param)
    cat = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)], early_stopping_rounds=10)
    score = mean_absolute_error(cat.predict(X_test), y_test)
    gc.collect()
    return score

studyCAT=optuna.create_study(direction='minimize')
studyCAT.optimize(lambda trial: objectiveCAT(trial, data[train_cols], data.target, seed=1004), n_trials=100)

[I 2022-11-10 07:08:16,786] A new study created in memory with name: no-name-9456d5d6-7639-42e6-8954-d12220955a4e
[I 2022-11-10 07:09:56,180] Trial 0 finished with value: 3.2614808863071545 and parameters: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 1004, 'od_wait': 10, 'learning_rate': 0.008330683375920309, 'depth': 11, 'reg_lambda': 83.18176529940038, 'random_strength': 45.992587677194535, 'min_data_in_leaf': 262, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.5431178221481548}. Best is trial 0 with value: 3.2614808863071545.
[I 2022-11-10 07:51:16,524] Trial 1 finished with value: 3.4368745900693916 and parameters: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 1004, 'od_wait': 10, 'learning_rate': 0.0019864594948658617, 'depth': 16, 'reg_lambda': 61.405942605995875, 'random_strength': 45.680308360355774, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 13, 'bagging_temperature': 4.062093017686514}. Best is trial 0 with value: 3.26148088630715

KeyboardInterrupt: 

In [ ]:
print('study.best_params:', studyCAT.best_trial.value)
print('Number of finished trials:', len(studyCAT.trials))
print('Best trial:', studyCAT.best_trial.params)
print('study.best_params:', studyCAT.best_params)

study.best_params: 2.942086457794878
Number of finished trials: 17
Best trial: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 1004, 'od_wait': 10, 'learning_rate': 0.04036000465315004, 'depth': 15, 'reg_lambda': 53.95201796934287, 'random_strength': 21.785778022759697, 'min_data_in_leaf': 108, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.017092835709441798}
study.best_params: {'task_type': 'GPU', 'iterations': 4000, 'random_state': 1004, 'od_wait': 10, 'learning_rate': 0.04036000465315004, 'depth': 15, 'reg_lambda': 53.95201796934287, 'random_strength': 21.785778022759697, 'min_data_in_leaf': 108, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.017092835709441798}


In [ ]:
# p = studyCAT.best_params
P = {'task_type': 'GPU', 'iterations': 4000, 'random_state': 1004, 'od_wait': 10, 'learning_rate': 0.04036000465315004, 'depth': 15, 'reg_lambda': 53.95201796934287, 'random_strength': 21.785778022759697, 'min_data_in_leaf': 108, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.017092835709441798}
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    models2.append(CatBoostRegressor(**p, silent=True, eval_metric='MAE'))
    models2[-1] = models2[-1].fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=5)
    pred = models2[-1].predict(X_val)
    mae.append(mean_absolute_error(pred, y_val))
    gc.collect()
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

0


Default metric period is 5 because MAE is/are not implemented for GPU


1


Default metric period is 5 because MAE is/are not implemented for GPU


2


...

3


Default metric period is 5 because MAE is/are not implemented for GPU


4


Default metric period is 5 because MAE is/are not implemented for GPU


분할의 MAE : [2.9347249698132294, 2.9155463018165593, 2.921472532716972, 2.9239239296433044, 2.9314708042244644, 2.9288166079427085, 2.9153689818015054, 2.922633045718397, 2.921168084309001, 2.928182032469698]
평균 MAE : 2.9243307290455838


## Tune Lgbm

In [ ]:
def objectiveLGBM(trial,X,y, seed):
    param = {
        'objective': trial.suggest_categorical("objective", ["regression"]), # 회귀
        'metric': trial.suggest_categorical("metric", ["mae"]),
        'random_state' : trial.suggest_int('random_state',seed, seed),
        'n_estimators': trial.suggest_int('num_iterations',3000, 3000),
        'verbosity':-1,
        'max_depth': trial.suggest_int('max_depth',15, 25),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('reg_lambda', 1e-1, 1),
        'num_leaves' : trial.suggest_int('num_leaves', 500, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 300),
    }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_test, label=y_test)
    
    # Add a callback for pruning.
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[early_stopping(5), log_evaluation(50)])
    
    best = gbm.best_iteration
    preds = gbm.predict(X_test, num_iteration=best)
    pred_labels = np.rint(preds)
    accuracy = mean_absolute_error(y_test, pred_labels)
    param['num_iterations'] = trial.suggest_int("num_iterations", best, best)
    gc.collect()
    return accuracy

studyLGBM=optuna.create_study(direction='minimize')
studyLGBM.optimize(lambda trial: objectiveLGBM(trial, data[train_cols], data.target, 726), n_trials=50) 

[I 2022-11-12 02:27:16,471] A new study created in memory with name: no-name-e5aa7a13-4ac4-4aab-827a-2c62bf1a4a6e
Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 10.1979
[100]	valid_0's l1: 7.96545
[150]	valid_0's l1: 6.38823
[200]	valid_0's l1: 5.29429
[250]	valid_0's l1: 4.55289
[300]	valid_0's l1: 4.05772
[350]	valid_0's l1: 3.7283
[400]	valid_0's l1: 3.51393
[450]	valid_0's l1: 3.37098
[500]	valid_0's l1: 3.27615
[550]	valid_0's l1: 3.21087
[600]	valid_0's l1: 3.16399
[650]	valid_0's l1: 3.12999
[700]	valid_0's l1: 3.10386
[750]	valid_0's l1: 3.08327
[800]	valid_0's l1: 3.06648
[850]	valid_0's l1: 3.05278
[900]	valid_0's l1: 3.04116
[950]	valid_0's l1: 3.03098
[1000]	valid_0's l1: 3.02196
[1050]	valid_0's l1: 3.01431
[1100]	valid_0's l1: 3.00754
[1150]	valid_0's l1: 3.00121
[1200]	valid_0's l1: 2.99568
[1250]	valid_0's l1: 2.99084
[1300]	valid_0's l1: 2.98616
[1350]	valid_0's l1: 2.98179
[1400]	valid_0's l1: 2.9781
[1450]	valid_0's l1: 2.97486
[1500]	valid_0's l1: 2.97181
[1550]	valid_0's l1: 2.96903
[1600]	valid_0's l1: 2.96634
[1650]	valid_0's l1: 2.9639
[1700]

[I 2022-11-12 02:39:23,407] Trial 0 finished with value: 2.9171743810768023 and parameters: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 18, 'learning_rate': 0.006019064252642324, 'subsample': 0.10703018114363878, 'reg_alpha': 0.49364877071087865, 'reg_lambda': 0.48054373310666443, 'num_leaves': 950, 'min_child_samples': 77}. Best is trial 0 with value: 2.9171743810768023.
Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 12.7564
[100]	valid_0's l1: 12.2296
[150]	valid_0's l1: 11.7266
[200]	valid_0's l1: 11.2493
[250]	valid_0's l1: 10.7971
[300]	valid_0's l1: 10.3678
[350]	valid_0's l1: 9.95912
[400]	valid_0's l1: 9.57279
[450]	valid_0's l1: 9.20591
[500]	valid_0's l1: 8.85906
[550]	valid_0's l1: 8.53157
[600]	valid_0's l1: 8.22044
[650]	valid_0's l1: 7.92615
[700]	valid_0's l1: 7.6469
[750]	valid_0's l1: 7.38303
[800]	valid_0's l1: 7.13403
[850]	valid_0's l1: 6.89918
[900]	valid_0's l1: 6.67733
[950]	valid_0's l1: 6.46882
[1000]	valid_0's l1: 6.27141
[1050]	valid_0's l1: 6.08467
[1100]	valid_0's l1: 5.90936
[1150]	valid_0's l1: 5.74402
[1200]	valid_0's l1: 5.58873
[1250]	valid_0's l1: 5.44251
[1300]	valid_0's l1: 5.30537
[1350]	valid_0's l1: 5.17652
[1400]	valid_0's l1: 5.05519
[1450]	valid_0's l1: 4.94091
[1500]	valid_0's l1: 4.83287
[1550]	valid_0's l1: 4.73201
[1600]	valid_0's l1: 4.63752
[1650]	valid_0's l1: 4.54885
[170

[I 2022-11-12 02:52:25,803] Trial 1 finished with value: 3.426126027129653 and parameters: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 17, 'learning_rate': 0.0009600925978668122, 'subsample': 0.4554799768035746, 'reg_alpha': 0.009717617968971748, 'reg_lambda': 0.5581149763721648, 'num_leaves': 534, 'min_child_samples': 5}. Best is trial 0 with value: 2.9171743810768023.
Found `n_estimators` in params. Will use it instead of argument


...

[I 2022-11-12 06:58:31,434] Trial 47 finished with value: 5.804093460150015 and parameters: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 15, 'learning_rate': 0.00035091531069511014, 'subsample': 0.19550965934986414, 'reg_alpha': 0.07315037750255508, 'reg_lambda': 0.8959447725313255, 'num_leaves': 738, 'min_child_samples': 55}. Best is trial 35 with value: 2.9032595468623454.
Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.0327
[100]	valid_0's l1: 2.96762
[150]	valid_0's l1: 2.94751
[200]	valid_0's l1: 2.93593
[250]	valid_0's l1: 2.9285
[300]	valid_0's l1: 2.92446
[350]	valid_0's l1: 2.92233
Early stopping, best iteration is:
[347]	valid_0's l1: 2.92215


Inconsistent parameter values for distribution with name "num_iterations"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 3000, 'high': 3000, 'step': 1}
[I 2022-11-12 06:59:33,412] Trial 48 finished with value: 2.9097957339779983 and parameters: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 17, 'learning_rate': 0.1315418721628845, 'subsample': 0.2341395464023071, 'reg_alpha': 0.01851326776747102, 'reg_lambda': 0.6975582199120106, 'num_leaves': 768, 'min_child_samples': 153}. Best is trial 35 with value: 2.9032595468623454.
Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.57185
[100]	valid_0's l1: 3.09268
[150]	valid_0's l1: 3.01469
[200]	valid_0's l1: 2.98086
[250]	valid_0's l1: 2.96332
[300]	valid_0's l1: 2.95124
[350]	valid_0's l1: 2.94198
[400]	valid_0's l1: 2.93444
[450]	valid_0's l1: 2.92867
[500]	valid_0's l1: 2.92521
[550]	valid_0's l1: 2.92284
[600]	valid_0's l1: 2.92087
[650]	valid_0's l1: 2.91836
[700]	valid_0's l1: 2.91676
[750]	valid_0's l1: 2.91522
Early stopping, best iteration is:
[748]	valid_0's l1: 2.91522


Inconsistent parameter values for distribution with name "num_iterations"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 3000, 'high': 3000, 'step': 1}
[I 2022-11-12 07:01:31,399] Trial 49 finished with value: 2.903678741926425 and parameters: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 20, 'learning_rate': 0.045538033725942485, 'subsample': 0.33047952076590903, 'reg_alpha': 0.1905582517005945, 'reg_lambda': 0.9942215955467435, 'num_leaves': 814, 'min_child_samples': 92}. Best is trial 35 with value: 2.9032595468623454.


In [ ]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

study.best_params: 2.9032595468623454
Number of finished trials: 50
Best trial: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 19, 'learning_rate': 0.04563210564471993, 'subsample': 0.3068082639597116, 'reg_alpha': 0.011007141059516187, 'reg_lambda': 0.8359525436017227, 'num_leaves': 720, 'min_child_samples': 95}
study.best_params: {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 19, 'learning_rate': 0.04563210564471993, 'subsample': 0.3068082639597116, 'reg_alpha': 0.011007141059516187, 'reg_lambda': 0.8359525436017227, 'num_leaves': 720, 'min_child_samples': 95}


In [ ]:
# lgbm
mae = []
models3 = []
p = {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 19, 'learning_rate': 0.04563210564471993, 'subsample': 0.3068082639597116, 'reg_alpha': 0.011007141059516187, 'reg_lambda': 0.8359525436017227, 'num_leaves': 720, 'min_child_samples': 95}
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_val, label=y_val)
    
    models3.append(lgb.train(p, dtrain, 10000, valid_sets=dvalid, callbacks=[early_stopping(5), log_evaluation(50)]))
    best = models3[-1].best_iteration
    pred = models3[-1].predict(X_val, num_iteration=best)
    mae.append(mean_absolute_error(pred, y_val))
    gc.collect()
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

0


Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050078 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3399
[LightGBM] [Info] Number of data points in the train set: 982832, number of used features: 27
[LightGBM] [Info] Start training from score 41.214508
Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.60822
[100]	valid_0's l1: 3.11427
[150]	valid_0's l1: 3.02975
[200]	valid_0's l1: 2.99463
[250]	valid_0's l1: 2.97398
[300]	valid_0's l1: 2.96147
[350]	valid_0's l1: 2.9522
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	valid_0's l1: 2.94465
[450]	valid_0's l1: 2.93805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3396
[LightGBM] [Info] Number of data points in the train set: 982833, number of used features: 27
[LightGBM] [Info] Start training from score 41.205076
Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.60758
[100]	valid_0's l1: 3.11332
[150]	valid_0's l1: 3.02918
[200]	valid_0's l1: 2.99019
[250]	valid_0's l1: 2.96998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	valid_0's l1: 2.95585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[350]	valid_0's l1: 2.94368
[400]	valid_0's l1: 2.93552
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3397
[LightGBM] [Info] Number of data points in the train set: 982833, number of used features: 27
[LightGBM] [Info] Start training from score 41.203697
Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.60181
[100]	valid_0's l1: 3.11343
[150]	valid_0's l1: 3.03172
[200]	valid_0's l1: 2.99348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	valid_0's l1: 2.97383
[300]	valid_0's l1: 2.96053
[350]	valid_0's l1: 2.95019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	valid_0's l1: 2.94219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[450]	

Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3395
[LightGBM] [Info] Number of data points in the train set: 982833, number of used features: 27
[LightGBM] [Info] Start training from score 41.208462
Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.60544
[100]	valid_0's l1: 3.10588
[150]	valid_0's l1: 3.02252
[200]	valid_0's l1: 2.98457
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	valid_0's l1: 2.96515
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	valid_0's l1: 2.95182
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[350]	valid_0's l1: 2.94215
[400]	valid_0's l1: 2.93462
[450]	

Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049576 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3398
[LightGBM] [Info] Number of data points in the train set: 982833, number of used features: 27
[LightGBM] [Info] Start training from score 41.207707
Training until validation scores don't improve for 5 rounds
[50]	valid_0's l1: 3.6072
[100]	valid_0's l1: 3.1153
[150]	valid_0's l1: 3.03089
[200]	valid_0's l1: 2.99419
[250]	valid_0's l1: 2.97489
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	valid_0's l1: 2.96203
[350]	valid_0's l1: 2.95191
[400]	valid_0's l1: 2.94397
[450]	valid_0's l1: 2.93766
[500]	valid_0's l1: 2.93333
[550]	valid_0's l1: 2.92889
[600]	valid_0's l1: 2.92503
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

In [ ]:
def objectiveLGBM(trial,X,y, seed):
    param = {
        'objective': trial.suggest_categorical("objective", ["regression"]), # 회귀
        'metric': trial.suggest_categorical("metric", ["mae"]),
        'random_state' : trial.suggest_int('random_state',seed, seed),
        'n_estimators': trial.suggest_int('num_iterations',3000, 3000),
        'verbosity':-1,
        'max_depth': trial.suggest_int('max_depth',15, 25),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('reg_lambda', 1e-1, 1),
        'num_leaves' : trial.suggest_int('num_leaves', 500, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 300),
    }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_test, label=y_test)
    
    # Add a callback for pruning.
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[early_stopping(5), log_evaluation(50)])
    
    best = gbm.best_iteration
    preds = gbm.predict(X_test, num_iteration=best)
    pred_labels = np.rint(preds)
    accuracy = mean_absolute_error(y_test, pred_labels)
    param['num_iterations'] = trial.suggest_int("num_iterations", best, best)
    gc.collect()
    return accuracy

studyLGBM=optuna.create_study(direction='minimize')
studyLGBM.optimize(lambda trial: objectiveLGBM(trial, data[train_cols], data.target, 1004), n_trials=50) 

In [ ]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

In [ ]:
# lgbm
mae = []
# p = {'objective': 'regression', 'metric': 'mae', 'random_state': 726, 'num_iterations': 3000, 'max_depth': 19, 'learning_rate': 0.04563210564471993, 'subsample': 0.3068082639597116, 'reg_alpha': 0.011007141059516187, 'reg_lambda': 0.8359525436017227, 'num_leaves': 720, 'min_child_samples': 95}
p = studyLGBM.best_params
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_val, label=y_val)
    
    models3.append(lgb.train(p, dtrain, 10000, valid_sets=dvalid, callbacks=[early_stopping(5), log_evaluation(50)]))
    best = models3[-1].best_iteration
    pred = models3[-1].predict(X_val, num_iteration=best)
    mae.append(mean_absolute_error(pred, y_val))
    gc.collect()
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

# road_name 별 모델
별로임

In [ ]:
cols = ['max/lane', 'start_longitude', 'road_rating', 'end_longitude',
       'start_latitude', 'end_latitude', 'maximum_speed_limit', 'base_hour',
       'start_node_name_trans', 'end_node_name_trans',
       'coordi_class', 'end_node_name', 'cos_base_hour', 'sin_base_hour',
       'weight_restricted', 'start_node_name', 'lane_count',
       'start_turn_restricted', 'road_type', 'month', 'day_of_week',
       'end_turn_restricted', '평균기온(°C)', 'day', 'is_rush', '평균 지면온도(°C)',
       '최대 풍속(m/s)', '평균 상대습도(%)', '합계 일조시간(hr)', '일강수량(mm)',
       'cos_day_of_week', 'DI', 'multi_linked', 'connect_code', '평균 풍속(m/s)',
       'sin_day_of_week', '평균 전운량(1/10)']

In [ ]:
cols

In [ ]:
rn = []
for index, i in enumerate(data.road_name.unique()):
    rn.append(data[data.road_name == i])

In [ ]:
pred = []
for index, df in tqdm(enumerate(rn)):
    reg = xgboost.XGBRegressor(**{'max_depth': 23, 'min_child_weight': 16, 'learning_rate': 0.08284207152264376, 'colsample_bytree': 0.6, 'lambda': 0.2853710092598808, 'alpha': 9.75864220081905, 'gamma': 0.06305836124110903, 'subsample': 0.9})
    reg.fit(df[cols], df.target)
    pred.append(reg.predict(test[cols]))
pred

In [ ]:
for index, df in tqdm(enumerate(rn)):
    reg = CatBoostRegressor(**{'learning_rate': 0.05293735024608017, 'depth': 14, 'od_wait': 761, 'reg_lambda': 90.99631271804222, 'random_strength': 11.822241871128384, 'min_data_in_leaf': 17, 'leaf_estimation_iterations': 4, 'bagging_temperature': 0.06503415171436373})
    reg.fit(df[cols], df.target)
    pred.append(reg.predict(test[cols]))
pred

In [ ]:
pred

In [ ]:
np.mean(pred, axis=0)

# seed ensemble

In [ ]:
data.columns

In [ ]:
train_cols = ['max/lane', 'start_longitude', 'road_rating', 'end_longitude',
       'start_latitude', 'end_latitude', 'maximum_speed_limit', 'base_hour',
       'road_name', 'class_start_node_name', 'class_end_node_name',
       'coordi_class', 'end_node_name', 'cos_base_hour', 'sin_base_hour',
       'weight_restricted', 'start_node_name', 'lane_count',
       'start_turn_restricted', 'road_type', 'month', 'day_of_week',
       'end_turn_restricted', '평균기온(°C)', 'day', 'is_rush', '평균 지면온도(°C)',
       '최대 풍속(m/s)', '평균 상대습도(%)', '합계 일조시간(hr)', '일강수량(mm)',
       'cos_day_of_week', 'DI', 'multi_linked', 'connect_code', '평균 풍속(m/s)',
       'sin_day_of_week', '평균 전운량(1/10)']
train_cols

## xgb1

In [ ]:
def objectiveXGB(trial,X,y, seed):
    param = {
        "n_estimators" : 300,
        'max_depth':trial.suggest_int('max_depth', 15, 25),
        'min_child_weight':trial.suggest_int('min_child_weight', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate',1e-3, 1e-1),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'gpu_id':0,
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] ),
        'random_state': seed,
        'seed' : seed,
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = xgboost.XGBRegressor(**param)
    xgb = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_absolute_error(xgb.predict(X_test), y_test)

    return score

studyXGB=optuna.create_study(direction='minimize')
studyXGB.optimize(lambda trial: objectiveXGB(trial, data[train_cols], data.target, 726), n_trials=50)

In [ ]:
print('study.best_params:', studyXGB.best_trial.value)
print('Number of finished trials:', len(studyXGB.trials))
print('Best trial:', studyXGB.best_trial.params)
print('study.best_params:', studyXGB.best_params)

In [ ]:
pred = []
# xgb_1 = xgboost.XGBRegressor(**{'max_depth': 25, 'min_child_weight': 12, 'learning_rate': 0.07186637213543264, 'colsample_bytree': 0.8, 'lambda': 7.613070848976136, 'alpha': 0.016468308993100617, 'gamma': 9.97309954008886, 'subsample': 0.9}
#                           ,tree_method='gpu_hist', gpu_id=0,random_state=726,seed=726,)
xgb_1 = xgboost.XGBRegressor(**{'max_depth': 25, 'min_child_weight': 12, 'learning_rate': 0.07186637213543264, 'colsample_bytree': 0.8, 'lambda': 7.613070848976136, 'alpha': 0.016468308993100617, 'gamma': 9.97309954008886, 'subsample': 0.9}
                         )
xgb_1.fit(data[train_cols], data.target)
pred.append(xgb_1.predict(test[train_cols]))

## xgb2

In [ ]:
def objectiveXGB(trial,X,y, seed):
    param = {
        "n_estimators" : 300,
        'max_depth':trial.suggest_int('max_depth', 15, 25),
        'min_child_weight':trial.suggest_int('min_child_weight', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate',1e-3, 1e-1),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'gpu_id':0,
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] ),
        'random_state': seed,
        'seed' : seed,
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = xgboost.XGBRegressor(**param)
    xgb = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_absolute_error(xgb.predict(X_test), y_test)

    return score

studyXGB=optuna.create_study(direction='minimize')
studyXGB.optimize(lambda trial: objectiveXGB(trial, data[train_cols], data.target, 1004), n_trials=50)

In [ ]:
print('study.best_params:', studyXGB.best_trial.value)
print('Number of finished trials:', len(studyXGB.trials))
print('Best trial:', studyXGB.best_trial.params)
print('study.best_params:', studyXGB.best_params)

In [ ]:
xgb_2 = xgboost.XGBRegressor(**{'max_depth': 22, 'min_child_weight': 10, 'learning_rate': 0.055602627356426085, 'colsample_bytree': 1.0, 'lambda': 0.48380593417918855, 'alpha': 0.1616709680444621, 'gamma': 0.1212236646464496, 'subsample': 0.9}
                             )
xgb_2.fit(data[train_cols], data.target)
pred.append(xgb_2.predict(test[train_cols]))

In [ ]:
pred

## cat1

In [ ]:
def objectiveCAT(trial,X,y,seed):
    param = {
        'silent':True,
        'task_type':'GPU',
        "iterations": 3000,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1),
        'depth': trial.suggest_int('depth', 1, 16),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = CatBoostRegressor(**param)
    cat = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_absolute_error(cat.predict(X_test), y_test)

    return score

studyCAT=optuna.create_study(direction='minimize')
studyCAT.optimize(lambda trial: objectiveCAT(trial, data[train_cols], data.target, 726), n_trials=50)

In [ ]:
print('study.best_params:', studyCAT.best_trial.value)
print('Number of finished trials:', len(studyCAT.trials))
print('Best trial:', studyCAT.best_trial.params)
print('study.best_params:', studyCAT.best_params)

In [ ]:
# cat_1 = CatBoostRegressor(**{'learning_rate': 0.11743446761516146, 'depth': 14, 'od_wait': 1786, 'reg_lambda': 90.25561982206659, 'random_strength': 45.80130032836945, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.018828355351985567}
#                           ,random_state=726,silent=True,task_type='GPU',iterations=3000,)
cat_1 = CatBoostRegressor(**{'learning_rate': 0.11743446761516146, 'depth': 14, 'od_wait': 1786, 'reg_lambda': 90.25561982206659, 'random_strength': 45.80130032836945, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.018828355351985567}
                          )
cat_1.fit(data[train_cols], data.target)
pred.append(cat_1.predict(test[train_cols]))

## cat2

In [ ]:
def objectiveCAT(trial,X,y,seed):
    param = {
        'silent':True,
        'task_type':'GPU',
        "iterations": 3000,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1),
        'depth': trial.suggest_int('depth', 1, 16),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = CatBoostRegressor(**param)
    cat = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_absolute_error(cat.predict(X_test), y_test)

    return score

studyCAT=optuna.create_study(direction='minimize')
studyCAT.optimize(lambda trial: objectiveCAT(trial, data[train_cols], data.target, 1004), n_trials=50)

In [ ]:
print('study.best_params:', studyCAT.best_trial.value)
print('Number of finished trials:', len(studyCAT.trials))
print('Best trial:', studyCAT.best_trial.params)
print('study.best_params:', studyCAT.best_params)

In [ ]:
# cat_2 = CatBoostRegressor(**{'learning_rate': 0.06939964400331083, 'depth': 15, 'od_wait': 2081, 'reg_lambda': 41.60873201624976, 'random_strength': 35.266008869545665, 'min_data_in_leaf': 2, 'leaf_estimation_iterations': 11, 'bagging_temperature': 0.010428752916620267}
#                           ,random_state=1004,silent=True,task_type='GPU',iterations=3000,)
cat_2 = CatBoostRegressor(**{'learning_rate': 0.06939964400331083, 'depth': 15, 'od_wait': 2081, 'reg_lambda': 41.60873201624976, 'random_strength': 35.266008869545665, 'min_data_in_leaf': 2, 'leaf_estimation_iterations': 11, 'bagging_temperature': 0.010428752916620267}
                          )
cat_2.fit(data[train_cols], data.target)
pred.append(cat_2.predict(test[train_cols]))

In [ ]:
pred

## lgbm1

In [ ]:
def objectiveLGBM(trial,X,y, seed):
    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'mae',
        'random_state' : seed,
        'max_depth': trial.suggest_int('max_depth',20, 30),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('reg_lambda', 1e-1, 1),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = lightgbm.LGBMRegressor(**param,)
    lgb = model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    score = mean_absolute_error(lgb.predict(X_test), y_test)
    gc.collect()
    return score

studyLGBM=optuna.create_study(direction='minimize')
studyLGBM.optimize(lambda trial: objectiveLGBM(trial, data[train_cols], data.target, 726), n_trials=100) 

In [ ]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

## lgbm2

In [ ]:
def objectiveLGBM(trial,X,y, seed):
    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'mae',
        'random_state' : seed,
        'max_depth': trial.suggest_int('max_depth',20, 30),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('reg_lambda', 1e-1, 1),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    model = lightgbm.LGBMRegressor(**param,)
    lgb = model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    score = mean_absolute_error(lgb.predict(X_test), y_test)
    gc.collect()
    return score

studyLGBM=optuna.create_study(direction='minimize')
studyLGBM.optimize(lambda trial: objectiveLGBM(trial, data[train_cols], data.target, 1004), n_trials=100) 

In [ ]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

In [ ]:
lgb_1 = lightgbm.LGBMRegressor(**{'max_depth': 22, 'learning_rate': 0.1078309669714809, 'n_estimators': 2824, 'subsample': 0.7412360854824448, 'reg_alpha': 0.0023481978955742005, 'reg_lambda': 0.7956120775834238, 'num_leaves': 887, 'min_child_samples': 1}
                               ,random_state=726)
lgb_1.fit(data[train_cols], data.target)
pred.append(lgb_1.predict(test[train_cols]))

lgb_2 = lightgbm.LGBMRegressor(**{'max_depth': 22, 'learning_rate': 0.026245494374002337, 'n_estimators': 2774, 'subsample': 0.21085833243338087, 'reg_alpha': 0.06049924002810861, 'reg_lambda': 0.7573169345428482, 'num_leaves': 776, 'min_child_samples': 123}
                               ,random_state=1004)
lgb_2.fit(data[train_cols], data.target)
pred.append(lgb_2.predict(test[train_cols]))

In [ ]:
pred

# KFold

In [ ]:
# cat
kfold = KFold(n_splits=15, shuffle=True, random_state=726)
mae = []
models = []
for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    models.append(lightgbm.LGBMRegressor(**studyLGBM.best_params))
    models[step] = models[step].fit(X_train, y_train)
    pred = models[step].predict(X_val)
    mae.append(mean_absolute_error(pred, y_val))
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

In [ ]:
# XGB
kfold = KFold(n_splits=15, shuffle=True, random_state=726)
mae = []
models = []
params = {'max_depth': 25, 'min_child_weight': 12, 'learning_rate': 0.07186637213543264, 'colsample_bytree': 0.8, 'lambda': 7.613070848976136, 'alpha': 0.016468308993100617, 'gamma': 9.97309954008886, 'subsample': 0.9}

for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    print(step)
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    models.append(xgboost.XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0))
    models[step] = models[step].fit(X_train, y_train)
    pred = models[step].predict(X_val)
    mae.append(mean_absolute_error(pred, y_val))
print("분할의 MAE :", mae)
print("평균 MAE :", np.mean(mae))

In [ ]:
# Tabnet
kfold = KFold(n_splits=15, shuffle=True, random_state=726)
mae = []
models = []

for step, (train_idx, val_idx) in enumerate(kfold.split(data[train_cols])):
    X_train, X_val = np.array(data[train_cols])[train_idx], np.array(data[train_cols])[val_idx]
    y_train, y_val = data.target[train_idx], data.target[val_idx]
    
    def objectiveLGBM(trial,X,y, seed):
    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'mae',
        'random_state' : seed,
        'max_depth': trial.suggest_int('max_depth',20, 30),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('reg_lambda', 1e-1, 1),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
    
    reg = TabNetRegressor(**tabnet_params)
    reg.fit(X_train=X_train, y_train=y_train, eval_set=[(X_val, y_val)],eval_metric=['mae'])
    score = mean_absolute_error(lgb.predict(X_test), y_test)
    gc.collect()
    return score

    studyTab=optuna.create_study(direction='minimize')
    studyTab.optimize(lambda trial: objectiveLGBM(trial, data[train_cols], data.target, 1004), n_trials=30) 
    
    print('study.best_params:', studyTab.best_trial.value)
    print('Number of finished trials:', len(studyTab.trials))
    print('Best trial:', studyTab.best_trial.params)
    print('study.best_params:', studyTab.best_params)

## tabnet

In [ ]:
len(data)//10

In [ ]:
def objectiveTab(trial,X,y, seed):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 8, 64)
    # n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    n_steps = 1
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(
                n_d=n_da, 
                n_a=n_da,
                n_steps=n_steps, 
                gamma=gamma,
                lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                mask_type=mask_type,
                n_shared=n_shared,
                scheduler_params=dict(mode="min",
                                   patience=trial.suggest_int("patienceScheduler",low=3,high=10), # changing sheduler patience to be lower than early stopping patience 
                                   min_lr=1e-5,
                                   factor=0.5,),
                scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                verbose=1,
                seed = seed,
                device_name='auto',
    )
    
    X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y).reshape(-1,1), test_size=0.2, random_state=seed)
    reg = TabNetRegressor(**tabnet_params)
    reg.fit(X_train=X_train, y_train=y_train, eval_set=[(X_test, y_test)], eval_metric=['mae'],patience=5,batch_size=122854, virtual_batch_size=122854, drop_last=False,)
    score = mean_absolute_error(reg.predict(X_test), y_test)
    gc.collect()
    return score

studyTab=optuna.create_study(direction='minimize')
studyTab.optimize(lambda trial: objectiveTab(trial, data[train_cols], data.target, 1004), n_trials=100) 

In [ ]:
preds = []
for i in tqdm(models):
    preds.append(i.predict(test[train_cols]))
    gc.collect()
    
for i in tqdm(models2):
    preds.append(i.predict(test[train_cols]))
    gc.collect()
base_targesum(preds)/len(preds)

# 제출

In [ ]:
def softmax(X):
    exp_a = np.exp(X)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

w = [2.873037527791208,2.8431248048852265,3.00412142570799,2.984232395637627]
w = [-i for i in w]
w = softmax(w)
w

In [ ]:
for index,i in enumerate(tqdm(models)):
    i.save_model(f'xgb_{index}.model')
    
for index,i in enumerate(tqdm(models2)):
    i.save_model(f'cat_{index}.model')
    
gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.73it/s]


2600

In [ ]:
preds = []
for i in tqdm(models):
    preds.append(i.predict(test[train_cols]))
    gc.collect()
    
for i in tqdm(models2):
    preds.append(i.predict(test[train_cols]))
    gc.collect()
    
for i in tqdm(models3):
    preds.append(i.predict(test[train_cols]))
    gc.collect()
preds

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:42<00:00, 32.50s/it]


[array([25.808437, 42.003468, 66.48897 , ..., 22.155373, 22.195568,
        48.002354], dtype=float32),
 array([25.907215, 42.133774, 66.644775, ..., 22.161844, 22.20553 ,
        48.37834 ], dtype=float32),
 array([26.108316, 42.080704, 66.7836  , ..., 21.682718, 21.374634,
        48.64545 ], dtype=float32),
 array([26.340723, 42.654755, 67.19887 , ..., 21.544422, 21.414742,
        48.59697 ], dtype=float32),
 array([25.510778, 42.46563 , 66.9296  , ..., 21.498579, 21.84393 ,
        48.364365], dtype=float32),
 array([25.958672, 42.241383, 67.20921 , ..., 22.203379, 21.888136,
        48.395794], dtype=float32),
 array([25.965723, 42.256897, 67.1631  , ..., 22.309628, 21.640924,
        48.168427], dtype=float32),
 array([26.279932, 42.364853, 66.99288 , ..., 21.902508, 21.571177,
        48.369423], dtype=float32),
 array([26.429064, 42.60878 , 67.678444, ..., 21.974394, 20.978968,
        48.603977], dtype=float32),
 array([25.862476, 42.48521 , 67.15141 , ..., 21.714869, 22.4911

In [ ]:
sum(preds)/len(preds)

array([25.97571393, 42.3504697 , 66.73832283, ..., 22.06201755,
       21.654797  , 48.37938535])

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub

,id,target
0,TEST_000000,0
1,TEST_000001,0
2,TEST_000002,0
3,TEST_000003,0
4,TEST_000004,0
...,...,...
291236,TEST_291236,0
291237,TEST_291237,0
291238,TEST_291238,0
291239,TEST_291239,0


In [ ]:
sub['target'] = sum(preds)/len(preds)
# sub['target'] = xgb.predict(test[train_cols])
sub.to_csv('xgb cat lgb seed 5fold.csv', index=False)

In [ ]:
sub

,id,target
0,TEST_000000,25.975714
1,TEST_000001,42.350470
2,TEST_000002,66.738323
3,TEST_000003,38.119701
4,TEST_000004,43.018256
...,...,...
291236,TEST_291236,47.039120
291237,TEST_291237,51.250450
291238,TEST_291238,22.062018
291239,TEST_291239,21.654797
